# Shift-Aware Time-Series Modelling and Bayesian Optimisation for Physics-Style Discovery

## Summary
This script presents a fully reproducible, decision-oriented data science workflow tailored to physics, astronomy, and mathematics applications where data are costly, distribution shift is common, and uncertainty must be operationally meaningful. Using simulated light-curve time series, the benchmark evaluates regression and classification under random splits, leakage-resistant group splits, and explicit out-of-distribution tests, showing that performance and calibration can appear optimistic under naive splitting but degrade under more realistic generalisation and shift. Reliability diagrams and risk–coverage curves demonstrate how calibration and selective prediction enable safer, uncertainty-aware deployment. The work then links prediction to experiment design by demonstrating Gaussian process Bayesian optimisation for an expensive simulator, and extends to multi-objective Bayesian optimisation that yields Pareto trade-offs prioritised by hypervolume contribution and ε-constraint decision rules. Overall, the study frames modern ML not only as prediction, but as reliable decision support under uncertainty and limited evaluation budgets.

# Reliable Scientific Data Science for Astronomy: Exoplanet Transit Detection + Parameter Estimation under Shift

## What this script demonstrates (end-to-end, reproducible):
    1) Physics-informed synthetic dataset generation (light curves with transits + stellar variability)
    2) Feature engineering (time-domain + FFT spectral features)
    3) Two tasks:
         - T1: Classification: "transit present?" (binary)
         - T2: Regression: estimate orbital period (days) when transit exists
    4) Realistic evaluation:
         - Random split (optimistic, leakage risk)
         - Group split by star_id (more realistic generalisation)
    5) OOD testing:
         - Changed noise and cadence (distribution shift)
    6) Uncertainty:
         - Probability calibration (isotonic/sigmoid)
         - Reliability diagrams + Brier score
         - Risk–coverage selective prediction curves (abstain on uncertain samples)
    7) Reproducible outputs:
         - Figures and CSV tables saved to --outdir

## Dependencies:
    numpy, pandas, matplotlib, scikit-learn

## Run:
    python uh_data_science_physics_astronomy_poc.py --outdir outputs


In [4]:
from __future__ import annotations

import os
import json
import math
import argparse
from dataclasses import dataclass
from typing import Dict, Tuple, Optional, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    f1_score,
    brier_score_loss,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.calibration import CalibratedClassifierCV, calibration_curve


# Utilities (reproducibility)

def set_seed(seed: int = 42) -> np.random.Generator:
    return np.random.default_rng(seed)


def ensure_outdir(outdir: str) -> str:
    os.makedirs(outdir, exist_ok=True)
    figdir = os.path.join(outdir, "figures")
    os.makedirs(figdir, exist_ok=True)
    tabdir = os.path.join(outdir, "tables")
    os.makedirs(tabdir, exist_ok=True)
    return outdir


def save_json(obj: Dict, path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)


# Synthetic astronomy generator

@dataclass
class SimConfig:
    n_stars: int = 300
    min_obs: int = 220
    max_obs: int = 320
    transit_fraction: float = 0.55  # fraction of stars with a planet transit
    duration_days: float = 30.0     # observation window length
    cadence_min: float = 0.08       # min sampling interval (days)
    cadence_max: float = 0.20       # max sampling interval (days)
    base_noise: float = 0.0009      # baseline photometric noise
    variability_amp: Tuple[float, float] = (0.0005, 0.0035)
    variability_freq: Tuple[float, float] = (0.02, 0.45)  # cycles per day
    transit_depth: Tuple[float, float] = (0.001, 0.02)    # fractional flux drop
    transit_width: Tuple[float, float] = (0.05, 0.35)     # days
    period_range: Tuple[float, float] = (1.2, 12.0)       # days
    jitter_time: float = 0.01                              # time jitter


def simulate_light_curve(
    rng: np.random.Generator,
    n_obs: int,
    duration_days: float,
    cadence_range: Tuple[float, float],
    noise_sigma: float,
    variability_amp_range: Tuple[float, float],
    variability_freq_range: Tuple[float, float],
    has_transit: bool,
    period_range: Tuple[float, float],
    transit_depth_range: Tuple[float, float],
    transit_width_range: Tuple[float, float],
    time_jitter: float,
) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """
    Simulate a "star" light curve:
        flux(t) = 1 + variability(t) - transit(t) + Gaussian noise

    Transit is modelled as repeated Gaussian dips at times around multiples of the orbital period.
    """
    # Irregular sampling times
    cadence = rng.uniform(cadence_range[0], cadence_range[1])
    t = np.arange(0.0, duration_days, cadence)
    if len(t) > n_obs:
        t = t[:n_obs]
    else:
        # if too few points, pad by re-sampling a slightly different cadence
        while len(t) < n_obs:
            cadence2 = rng.uniform(cadence_range[0], cadence_range[1])
            t2 = np.arange(0.0, duration_days, cadence2)
            t = np.unique(np.concatenate([t, t2]))
            t = t[:n_obs]

    # Add jitter
    t = t + rng.normal(0.0, time_jitter, size=t.shape)
    t = np.clip(t, 0.0, duration_days)
    t = np.sort(t)

    # Stellar variability (sinusoid + second harmonic)
    amp = rng.uniform(*variability_amp_range)
    freq = rng.uniform(*variability_freq_range)
    phase = rng.uniform(0, 2 * np.pi)
    variability = amp * np.sin(2 * np.pi * freq * t + phase) + 0.35 * amp * np.sin(2 * np.pi * (2*freq) * t + 0.7*phase)

    # Transit parameters (if present)
    if has_transit:
        period = rng.uniform(*period_range)
        depth = rng.uniform(*transit_depth_range)
        width = rng.uniform(*transit_width_range)
        t0 = rng.uniform(0.0, period)  # random epoch within first period

        # Create dips: sum of Gaussians at each transit center
        # (simple but effective "physics-ish" proxy for dips)
        centers = t0 + np.arange(-2, int(duration_days / period) + 3) * period
        transit = np.zeros_like(t)
        for c in centers:
            transit += depth * np.exp(-0.5 * ((t - c) / width) ** 2)
    else:
        period = np.nan
        depth = np.nan
        width = np.nan
        t0 = np.nan
        transit = np.zeros_like(t)

    # Observed flux
    noise = rng.normal(0.0, noise_sigma, size=t.shape)
    flux = 1.0 + variability - transit + noise

    meta = {
        "period": period,
        "depth": depth,
        "width": width,
        "variability_amp": amp,
        "variability_freq": freq,
        "noise_sigma": noise_sigma,
    }
    return t, flux, meta


def simulate_dataset(rng: np.random.Generator, cfg: SimConfig) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns:
        curves_df: long-form time series with columns [star_id, t, flux]
        labels_df: per-star table with targets/metadata
    """
    curves = []
    labels = []

    for star_id in range(cfg.n_stars):
        n_obs = int(rng.integers(cfg.min_obs, cfg.max_obs + 1))
        has_transit = rng.random() < cfg.transit_fraction

        t, flux, meta = simulate_light_curve(
            rng=rng,
            n_obs=n_obs,
            duration_days=cfg.duration_days,
            cadence_range=(cfg.cadence_min, cfg.cadence_max),
            noise_sigma=cfg.base_noise * rng.uniform(0.85, 1.25),
            variability_amp_range=cfg.variability_amp,
            variability_freq_range=cfg.variability_freq,
            has_transit=has_transit,
            period_range=cfg.period_range,
            transit_depth_range=cfg.transit_depth,
            transit_width_range=cfg.transit_width,
            time_jitter=cfg.jitter_time,
        )

        curves.append(pd.DataFrame({"star_id": star_id, "t": t, "flux": flux}))
        labels.append(
            {
                "star_id": star_id,
                "has_transit": int(has_transit),
                "period_days": float(meta["period"]) if has_transit else np.nan,
                "transit_depth": float(meta["depth"]) if has_transit else np.nan,
                "transit_width": float(meta["width"]) if has_transit else np.nan,
                "variability_amp": float(meta["variability_amp"]),
                "variability_freq": float(meta["variability_freq"]),
                "noise_sigma": float(meta["noise_sigma"]),
            }
        )

    curves_df = pd.concat(curves, ignore_index=True)
    labels_df = pd.DataFrame(labels).sort_values("star_id").reset_index(drop=True)
    return curves_df, labels_df


# Feature engineering

def extract_features_for_star(t: np.ndarray, flux: np.ndarray) -> Dict[str, float]:
    """
    Physics/data-science friendly features:
        - time-domain robust stats
        - simple "transit-likeness" metrics
        - FFT spectral features (dominant frequency power)
    """
    # Basic stats
    x = flux.astype(np.float64)
    median = float(np.median(x))
    mad = float(np.median(np.abs(x - median))) + 1e-12  # robust scale

    # Detrend lightly (remove median)
    y = x - median

    # Time-domain moments
    std = float(np.std(y))
    p01 = float(np.quantile(y, 0.01))
    p05 = float(np.quantile(y, 0.05))
    p95 = float(np.quantile(y, 0.95))
    p99 = float(np.quantile(y, 0.99))

    # "Dip" emphasis: how strong are negative excursions?
    neg_tail = float(np.mean(y[y < np.quantile(y, 0.10)])) if np.any(y < np.quantile(y, 0.10)) else 0.0
    dip_score = float((np.abs(p01) + np.abs(p05)) / (std + 1e-9))

    # Autocorrelation at small lag (rough periodicity signal)
    # Interpolate to uniform grid for rough ACF and FFT
    # Use a fixed grid length
    n_grid = 256
    t_min, t_max = float(np.min(t)), float(np.max(t))
    if t_max - t_min < 1e-6:
        # degenerate
        return {
            "median": median,
            "mad": mad,
            "std": std,
            "p01": p01,
            "p05": p05,
            "p95": p95,
            "p99": p99,
            "neg_tail": neg_tail,
            "dip_score": dip_score,
            "acf1": 0.0,
            "acf5": 0.0,
            "fft_peak_freq": 0.0,
            "fft_peak_power": 0.0,
            "fft_power_ratio": 0.0,
        }

    grid_t = np.linspace(t_min, t_max, n_grid)
    grid_y = np.interp(grid_t, t, y)

    # ACF at lag 1 and 5
    def autocorr(a: np.ndarray, lag: int) -> float:
        if lag >= len(a):
            return 0.0
        a0 = a[:-lag]
        a1 = a[lag:]
        denom = (np.std(a0) * np.std(a1) + 1e-12)
        return float(np.mean((a0 - np.mean(a0)) * (a1 - np.mean(a1))) / denom)

    acf1 = autocorr(grid_y, 1)
    acf5 = autocorr(grid_y, 5)

    # FFT
    # Remove mean, use rfft
    gy = grid_y - np.mean(grid_y)
    fft = np.fft.rfft(gy)
    power = (np.abs(fft) ** 2).astype(np.float64)
    freqs = np.fft.rfftfreq(n_grid, d=(grid_t[1] - grid_t[0]))

    # Ignore DC component
    if len(power) > 1:
        power_no_dc = power.copy()
        power_no_dc[0] = 0.0
        peak_idx = int(np.argmax(power_no_dc))
        fft_peak_freq = float(freqs[peak_idx])
        fft_peak_power = float(power_no_dc[peak_idx])
        total_power = float(np.sum(power_no_dc) + 1e-12)
        fft_power_ratio = float(fft_peak_power / total_power)
    else:
        fft_peak_freq, fft_peak_power, fft_power_ratio = 0.0, 0.0, 0.0

    return {
        "median": median,
        "mad": mad,
        "std": std,
        "p01": p01,
        "p05": p05,
        "p95": p95,
        "p99": p99,
        "neg_tail": neg_tail,
        "dip_score": dip_score,
        "acf1": acf1,
        "acf5": acf5,
        "fft_peak_freq": fft_peak_freq,
        "fft_peak_power": fft_peak_power,
        "fft_power_ratio": fft_power_ratio,
    }


def build_feature_table(curves_df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert long-form time series to per-star feature table.
    """
    feats = []
    for star_id, sub in curves_df.groupby("star_id", sort=True):
        t = sub["t"].to_numpy()
        flux = sub["flux"].to_numpy()
        f = extract_features_for_star(t, flux)
        f["star_id"] = int(star_id)
        feats.append(f)

    feat_df = pd.DataFrame(feats).sort_values("star_id").reset_index(drop=True)
    return feat_df


# Evaluation helpers

def classification_metrics(y_true: np.ndarray, proba: np.ndarray, thresh: float = 0.5) -> Dict[str, float]:
    y_pred = (proba >= thresh).astype(int)
    return {
        "roc_auc": float(roc_auc_score(y_true, proba)),
        "avg_precision": float(average_precision_score(y_true, proba)),
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "f1": float(f1_score(y_true, y_pred)),
        "brier": float(brier_score_loss(y_true, proba)),
    }


def regression_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    return {
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "rmse": float(rmse),
        "r2": float(r2_score(y_true, y_pred)),
    }


def plot_reliability(y_true: np.ndarray, proba: np.ndarray, title: str, outpath: str) -> None:
    frac_pos, mean_pred = calibration_curve(y_true, proba, n_bins=10, strategy="uniform")

    plt.figure(figsize=(6.4, 5.2))
    plt.plot(mean_pred, frac_pos, marker="o")
    plt.plot([0, 1], [0, 1])
    plt.xlabel("Mean predicted probability")
    plt.ylabel("Fraction of positives")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()


def risk_coverage_curve(y_true: np.ndarray, proba: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Selective prediction: abstain on low-confidence points.
    Risk = 1 - accuracy on retained subset
    Coverage = fraction retained
    """
    # confidence = max(p, 1-p)
    conf = np.maximum(proba, 1.0 - proba)
    order = np.argsort(-conf)  # high confidence first

    y_true_ord = y_true[order]
    proba_ord = proba[order]
    pred_ord = (proba_ord >= 0.5).astype(int)

    risks = []
    coverages = []
    for k in range(10, len(y_true_ord) + 1, max(10, len(y_true_ord) // 60)):
        yt = y_true_ord[:k]
        yp = pred_ord[:k]
        acc = accuracy_score(yt, yp)
        risk = 1.0 - acc
        coverage = k / len(y_true_ord)
        risks.append(risk)
        coverages.append(coverage)

    return np.array(coverages), np.array(risks)


def plot_risk_coverage(y_true: np.ndarray, proba: np.ndarray, title: str, outpath: str) -> None:
    cov, risk = risk_coverage_curve(y_true, proba)
    plt.figure(figsize=(6.4, 5.2))
    plt.plot(cov, risk, marker="o")
    plt.xlabel("Coverage (fraction predicted)")
    plt.ylabel("Risk (1 - accuracy)")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()


# OOD dataset generator

def simulate_ood_dataset(rng: np.random.Generator, base_cfg: SimConfig) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    OOD shift:
        - higher noise
        - slightly different cadence regime
        - stronger variability amplitude distribution
    """
    cfg = SimConfig(**vars(base_cfg))
    cfg.base_noise *= 2.1
    cfg.cadence_min *= 0.7
    cfg.cadence_max *= 1.35
    cfg.variability_amp = (base_cfg.variability_amp[0] * 1.2, base_cfg.variability_amp[1] * 1.6)

    return simulate_dataset(rng, cfg)


# Main experiment

def run_experiment(
    outdir: str,
    seed: int = 42,
    n_stars: int = 300,
    model_kind: str = "rf",
) -> None:
    """
    model_kind:
        - "lr": LogisticRegression + Ridge
        - "rf": RandomForest
    """
    rng = set_seed(seed)
    ensure_outdir(outdir)

    # 1) Simulate train-like dataset
    cfg = SimConfig(n_stars=n_stars)
    curves_df, labels_df = simulate_dataset(rng, cfg)

    # Save raw dataset summaries
    labels_df.to_csv(os.path.join(outdir, "tables", "labels_table.csv"), index=False)
    curves_df.head(2000).to_csv(os.path.join(outdir, "tables", "lightcurves_preview.csv"), index=False)

    # 2) Feature extraction
    feat_df = build_feature_table(curves_df)
    data = feat_df.merge(labels_df, on="star_id", how="inner")

    # Define targets
    y_cls = data["has_transit"].to_numpy().astype(int)
    groups = data["star_id"].to_numpy().astype(int)

    # Regression only for positive class
    pos_mask = data["has_transit"].to_numpy().astype(int) == 1
    y_reg = data.loc[pos_mask, "period_days"].to_numpy().astype(float)

    # Feature matrix
    feature_cols = [
        "mad", "std", "p01", "p05", "p95", "p99",
        "neg_tail", "dip_score", "acf1", "acf5",
        "fft_peak_freq", "fft_peak_power", "fft_power_ratio"
    ]
    X = data[feature_cols].to_numpy().astype(float)
    X_pos = data.loc[pos_mask, feature_cols].to_numpy().astype(float)

    # 3) Two split strategies:
    #    A) Random split (optimistic)
    X_tr_r, X_te_r, y_tr_r, y_te_r = train_test_split(X, y_cls, test_size=0.25, random_state=seed, stratify=y_cls)

    #    B) Group split by star_id (more realistic generalisation)
    gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    tr_idx, te_idx = next(gss.split(X, y_cls, groups=groups))
    X_tr_g, X_te_g = X[tr_idx], X[te_idx]
    y_tr_g, y_te_g = y_cls[tr_idx], y_cls[te_idx]

    # 4) Choose model
    if model_kind == "lr":
        base_clf = Pipeline(
            steps=[
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(max_iter=2000, class_weight="balanced")),
            ]
        )
        base_reg = Pipeline(
            steps=[
                ("scaler", StandardScaler()),
                ("reg", Ridge(alpha=1.0)),
            ]
        )
    elif model_kind == "rf":
        base_clf = RandomForestClassifier(
            n_estimators=600,
            random_state=seed,
            max_depth=None,
            min_samples_leaf=3,
            class_weight="balanced_subsample",
        )
        base_reg = RandomForestRegressor(
            n_estimators=800,
            random_state=seed,
            max_depth=None,
            min_samples_leaf=3,
        )
    else:
        raise ValueError("model_kind must be one of: 'lr', 'rf'")

    # 5) Fit + calibrate classifier on each split
    # Use calibration to showcase reliability
    # Note: CalibratedClassifierCV expects an unfitted base estimator
    def fit_calibrated_classifier(X_train: np.ndarray, y_train: np.ndarray) -> CalibratedClassifierCV:
        cal = CalibratedClassifierCV(estimator=base_clf, method="isotonic", cv=3)
        cal.fit(X_train, y_train)
        return cal

    clf_r = fit_calibrated_classifier(X_tr_r, y_tr_r)
    clf_g = fit_calibrated_classifier(X_tr_g, y_tr_g)

    proba_r = clf_r.predict_proba(X_te_r)[:, 1]
    proba_g = clf_g.predict_proba(X_te_g)[:, 1]

    metrics_cls_random = classification_metrics(y_te_r, proba_r)
    metrics_cls_group = classification_metrics(y_te_g, proba_g)

    # 6) Regression model for period estimation (positives only)
    # We'll create regression train/test using same group split logic within positives
    # Random split (positives)
    Xp_tr_r, Xp_te_r, yr_tr_r, yr_te_r = train_test_split(
        X_pos, y_reg, test_size=0.25, random_state=seed
    )

    # Group split within positives
    pos_star_ids = data.loc[pos_mask, "star_id"].to_numpy().astype(int)
    gss2 = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    trp_idx, tep_idx = next(gss2.split(X_pos, y_reg, groups=pos_star_ids))
    Xp_tr_g, Xp_te_g = X_pos[trp_idx], X_pos[tep_idx]
    yr_tr_g, yr_te_g = y_reg[trp_idx], y_reg[tep_idx]

    # Fit regression models
    reg_r = base_reg
    reg_g = base_reg
    reg_r.fit(Xp_tr_r, yr_tr_r)
    reg_g.fit(Xp_tr_g, yr_tr_g)

    pred_pr = reg_r.predict(Xp_te_r)
    pred_pg = reg_g.predict(Xp_te_g)

    metrics_reg_random = regression_metrics(yr_te_r, pred_pr)
    metrics_reg_group = regression_metrics(yr_te_g, pred_pg)

    # 7) OOD dataset evaluation
    curves_ood, labels_ood = simulate_ood_dataset(rng, cfg)
    feat_ood = build_feature_table(curves_ood)
    data_ood = feat_ood.merge(labels_ood, on="star_id", how="inner")

    X_ood = data_ood[feature_cols].to_numpy().astype(float)
    y_ood = data_ood["has_transit"].to_numpy().astype(int)
    proba_ood_r = clf_r.predict_proba(X_ood)[:, 1]
    proba_ood_g = clf_g.predict_proba(X_ood)[:, 1]

    metrics_ood_from_random = classification_metrics(y_ood, proba_ood_r)
    metrics_ood_from_group = classification_metrics(y_ood, proba_ood_g)

    # 8) Save metrics JSON
    report = {
        "config": vars(cfg),
        "seed": seed,
        "n_stars": n_stars,
        "model_kind": model_kind,
        "classification": {
            "random_split": metrics_cls_random,
            "group_split": metrics_cls_group,
            "ood_from_random_model": metrics_ood_from_random,
            "ood_from_group_model": metrics_ood_from_group,
        },
        "regression_period_days": {
            "random_split": metrics_reg_random,
            "group_split": metrics_reg_group,
        },
    }
    save_json(report, os.path.join(outdir, "metrics_report.json"))

    # 9) Produce figures
    figdir = os.path.join(outdir, "figures")

    # Example light curves (4 random stars)
    example_ids = rng.choice(labels_df["star_id"].to_numpy(), size=4, replace=False)
    plt.figure(figsize=(10, 7))
    for i, sid in enumerate(example_ids, start=1):
        sub = curves_df[curves_df["star_id"] == sid].sort_values("t")
        plt.subplot(2, 2, i)  # minimal multi-panel just for examples
        plt.plot(sub["t"], sub["flux"], linewidth=1.0)
        lab = labels_df.loc[labels_df["star_id"] == sid, "has_transit"].iloc[0]
        plt.title(f"Star {sid} | transit={lab}")
        plt.xlabel("Time (days)")
        plt.ylabel("Flux")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "example_lightcurves.png"), dpi=200)
    plt.close()

    # Calibration plots
    plot_reliability(
        y_te_r, proba_r,
        title="Reliability Diagram, Random Split (Calibrated)",
        outpath=os.path.join(figdir, "reliability_random_split.png"),
    )
    plot_reliability(
        y_te_g, proba_g,
        title="Reliability Diagram, Group Split (Calibrated)",
        outpath=os.path.join(figdir, "reliability_group_split.png"),
    )
    plot_reliability(
        y_ood, proba_ood_g,
        title="Reliability Diagram, OOD Test (Group-Trained Calibrated)",
        outpath=os.path.join(figdir, "reliability_ood.png"),
    )

    # Risk–coverage curves
    plot_risk_coverage(
        y_te_r, proba_r,
        title="Risk–Coverage, Random Split",
        outpath=os.path.join(figdir, "risk_coverage_random.png"),
    )
    plot_risk_coverage(
        y_te_g, proba_g,
        title="Risk–Coverage, Group Split",
        outpath=os.path.join(figdir, "risk_coverage_group.png"),
    )
    plot_risk_coverage(
        y_ood, proba_ood_g,
        title="Risk–Coverage, OOD (Group-Trained)",
        outpath=os.path.join(figdir, "risk_coverage_ood.png"),
    )

    # Performance summary bar chart
    plt.figure(figsize=(8.2, 5.2))
    names = ["Random", "Group", "OOD(Random)", "OOD(Group)"]
    aucs = [
        report["classification"]["random_split"]["roc_auc"],
        report["classification"]["group_split"]["roc_auc"],
        report["classification"]["ood_from_random_model"]["roc_auc"],
        report["classification"]["ood_from_group_model"]["roc_auc"],
    ]
    plt.bar(names, aucs)
    plt.ylim(0.0, 1.0)
    plt.ylabel("ROC-AUC")
    plt.title("Transit Detection Performance under Split Strategy and Shift")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "auc_summary.png"), dpi=200)
    plt.close()

    # Regression scatter plots
    plt.figure(figsize=(6.4, 5.2))
    plt.scatter(yr_te_g, pred_pg, s=18)
    lo = min(np.min(yr_te_g), np.min(pred_pg))
    hi = max(np.max(yr_te_g), np.max(pred_pg))
    plt.plot([lo, hi], [lo, hi])
    plt.xlabel("True period (days)")
    plt.ylabel("Predicted period (days)")
    plt.title("Period Estimation, Group Split")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "period_regression_group.png"), dpi=200)
    plt.close()

    # 10) Save a concise results table
    rows = []
    for k, v in report["classification"].items():
        rows.append({"setting": k, **v})
    cls_table = pd.DataFrame(rows)
    cls_table.to_csv(os.path.join(outdir, "tables", "classification_metrics.csv"), index=False)

    reg_table = pd.DataFrame(
        [
            {"setting": "random_split", **report["regression_period_days"]["random_split"]},
            {"setting": "group_split", **report["regression_period_days"]["group_split"]},
        ]
    )
    reg_table.to_csv(os.path.join(outdir, "tables", "regression_metrics.csv"), index=False)

    # Print a readable summary
    print("\n=== Data Science for Astronomy PoC ===")
    print(f"Outputs saved to: {os.path.abspath(outdir)}\n")
    print("Classification (Transit detection)")
    print("  Random split:", report["classification"]["random_split"])
    print("  Group split :", report["classification"]["group_split"])
    print("  OOD (rand)  :", report["classification"]["ood_from_random_model"])
    print("  OOD (group) :", report["classification"]["ood_from_group_model"])
    print("\nRegression (Period estimation, positives only)")
    print("  Random split:", report["regression_period_days"]["random_split"])
    print("  Group split :", report["regression_period_days"]["group_split"])
    print("\nKey figures written to outputs/figures/")


def make_argparser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="UH Data Science (Physics/Astronomy/Math) Proof-of-Concept")
    p.add_argument("--outdir", type=str, default="uh_poc_outputs", help="Output directory for figures/tables")
    p.add_argument("--seed", type=int, default=42, help="Random seed")
    p.add_argument("--n_stars", type=int, default=300, help="Number of synthetic stars")
    p.add_argument("--model", type=str, default="rf", choices=["rf", "lr"], help="Model type")
    return p


def main():
    parser = make_argparser()
    args, _ = parser.parse_known_args()  # Robust in Jupyter + terminal
    run_experiment(outdir=args.outdir, seed=args.seed, n_stars=args.n_stars, model_kind=args.model)


if __name__ == "__main__":
    main()


=== Data Science for Astronomy PoC ===
Outputs saved to: /Users/petalc01/Data Science Hertfordshire/uh_poc_outputs

Classification (Transit detection)
  Random split: {'roc_auc': 0.851017441860465, 'avg_precision': 0.9107412808532469, 'accuracy': 0.7733333333333333, 'f1': 0.7733333333333333, 'brier': 0.16322909565075674}
  Group split : {'roc_auc': 0.904074074074074, 'avg_precision': 0.9347588427951816, 'accuracy': 0.8666666666666667, 'f1': 0.8809523809523809, 'brier': 0.10766760867462419}
  OOD (rand)  : {'roc_auc': 0.8049510701997408, 'avg_precision': 0.8059992001204431, 'accuracy': 0.5733333333333334, 'f1': 0.7023255813953488, 'brier': 0.35099630482449795}
  OOD (group) : {'roc_auc': 0.6715224094016712, 'avg_precision': 0.6490832254924903, 'accuracy': 0.5766666666666667, 'f1': 0.703962703962704, 'brier': 0.35228743917735567}

Regression (Period estimation, positives only)
  Random split: {'mae': 2.614105589001194, 'rmse': 2.970435500428323, 'r2': 0.24950818565430355}
  Group split 

# Inverse Problems in Physics with Reliable Data Science: Damped Harmonic Oscillator Parameter Recovery (γ, ω0) under Noise and Shift

## What this script demonstrates (end-to-end, reproducible):
    1) Physics-informed synthetic dataset generation:
         x(t) = A * exp(-γ t) * cos(ω_d t + φ) + noise
         ω_d = sqrt(ω0^2 - γ^2), underdamped regime
    2) Feature engineering:
         - time-domain stats
         - FFT peak frequency and power ratio
         - envelope decay estimation (log amplitude slope)
         - autocorrelation features
    3) Two modelling approaches:
         - Mechanistic baseline estimators (physics-inspired)
         - ML models: RandomForestRegressor and GaussianProcessRegressor (uncertainty intervals)
    4) Robust evaluation:
         - random split
         - group split (system_id grouping) to reduce leakage bias
    5) OOD testing:
         - increased noise, altered sampling cadence
    6) Uncertainty:
         - GPR predictive mean and standard deviation
         - coverage of 95% predictive interval
    7) Reproducible outputs:
         - figures, tables, metrics JSON saved to --outdir

## Dependencies:
    numpy, pandas, matplotlib, scikit-learn

## Run:
    python uh_data_science_physics_math_poc.py --outdir outputs_physics_math

In [27]:
from __future__ import annotations

import os
import json
import math
import argparse
from dataclasses import dataclass
from typing import Dict, Tuple, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from sklearn.exceptions import ConvergenceWarning

# Silence ONLY the specific warning you reported (optional but clean).
# Keeping this is harmless even after fixing the kernel bounds.

warnings.filterwarnings(
    "ignore",
    message=r".*parameter k2__noise_level is close to the specified lower bound.*",
    category=ConvergenceWarning,
    module=r"sklearn\.gaussian_process\.kernels",
)

# Reproducibility utilities

def set_seed(seed: int = 42) -> np.random.Generator:
    return np.random.default_rng(seed)


def ensure_outdir(outdir: str) -> str:
    os.makedirs(outdir, exist_ok=True)
    os.makedirs(os.path.join(outdir, "figures"), exist_ok=True)
    os.makedirs(os.path.join(outdir, "tables"), exist_ok=True)
    return outdir


def save_json(obj: Dict, path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

# Physics generator

@dataclass
class OscillatorConfig:
    n_systems: int = 400
    duration: float = 8.0        # seconds
    min_obs: int = 250
    max_obs: int = 420
    noise_sigma: float = 0.03
    cadence_min: float = 0.010   # seconds
    cadence_max: float = 0.030   # seconds

    # parameter ranges
    A_range: Tuple[float, float] = (0.8, 2.2)
    gamma_range: Tuple[float, float] = (0.10, 1.20)  # damping γ
    omega0_range: Tuple[float, float] = (3.5, 14.0)  # natural ω0
    phi_range: Tuple[float, float] = (0.0, 2*np.pi)


def simulate_damped_oscillator(
    rng: np.random.Generator,
    n_obs: int,
    duration: float,
    cadence_range: Tuple[float, float],
    noise_sigma: float,
    A_range: Tuple[float, float],
    gamma_range: Tuple[float, float],
    omega0_range: Tuple[float, float],
    phi_range: Tuple[float, float],
) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """
    Underdamped oscillator:
        x(t) = A e^{-γt} cos(ω_d t + φ) + ε
        ω_d = sqrt(ω0^2 - γ^2), enforce ω0 > γ
    """
    # sample parameters, enforce underdamped regime
    A = rng.uniform(*A_range)
    gamma = rng.uniform(*gamma_range)

    # guarantee omega0 > gamma with margin
    omega0 = rng.uniform(max(omega0_range[0], gamma + 0.5), omega0_range[1])
    phi = rng.uniform(*phi_range)

    omega_d = math.sqrt(max(omega0**2 - gamma**2, 1e-12))

    # irregular sampling
    cadence = rng.uniform(*cadence_range)
    t = np.arange(0.0, duration, cadence)
    if len(t) > n_obs:
        t = t[:n_obs]
    else:
        while len(t) < n_obs:
            cadence2 = rng.uniform(*cadence_range)
            t2 = np.arange(0.0, duration, cadence2)
            t = np.unique(np.concatenate([t, t2]))
            t = t[:n_obs]
    t = np.sort(t)

    # signal + noise
    x_clean = A * np.exp(-gamma * t) * np.cos(omega_d * t + phi)
    x = x_clean + rng.normal(0.0, noise_sigma, size=t.shape)

    meta = {
        "A": float(A),
        "gamma": float(gamma),
        "omega0": float(omega0),
        "omega_d": float(omega_d),
        "phi": float(phi),
        "noise_sigma": float(noise_sigma),
    }
    return t, x, meta


def simulate_dataset(rng: np.random.Generator, cfg: OscillatorConfig) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns:
        series_df: long-form time series table [system_id, t, x]
        labels_df: per-system parameters [system_id, gamma, omega0, ...]
    """
    series = []
    labels = []

    for system_id in range(cfg.n_systems):
        n_obs = int(rng.integers(cfg.min_obs, cfg.max_obs + 1))
        t, x, meta = simulate_damped_oscillator(
            rng=rng,
            n_obs=n_obs,
            duration=cfg.duration,
            cadence_range=(cfg.cadence_min, cfg.cadence_max),
            noise_sigma=cfg.noise_sigma * rng.uniform(0.85, 1.35),
            A_range=cfg.A_range,
            gamma_range=cfg.gamma_range,
            omega0_range=cfg.omega0_range,
            phi_range=cfg.phi_range,
        )

        series.append(pd.DataFrame({"system_id": system_id, "t": t, "x": x}))
        labels.append({"system_id": system_id, **meta})

    return pd.concat(series, ignore_index=True), pd.DataFrame(labels).sort_values("system_id").reset_index(drop=True)


def simulate_ood_dataset(rng: np.random.Generator, base_cfg: OscillatorConfig) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    OOD shift:
        - higher noise
        - different cadence regime
    """
    cfg = OscillatorConfig(**vars(base_cfg))
    cfg.noise_sigma *= 2.2
    cfg.cadence_min *= 0.65
    cfg.cadence_max *= 1.6
    return simulate_dataset(rng, cfg)

# Feature engineering (scientific)

def _autocorr(x: np.ndarray, lag: int) -> float:
    if lag <= 0 or lag >= len(x):
        return 0.0
    x0 = x[:-lag]
    x1 = x[lag:]
    denom = (np.std(x0) * np.std(x1) + 1e-12)
    return float(np.mean((x0 - np.mean(x0)) * (x1 - np.mean(x1))) / denom)


def extract_features(t: np.ndarray, x: np.ndarray) -> Dict[str, float]:
    """
    Features designed to recover oscillator parameters robustly:
        - robust stats
        - FFT peak frequency ~ ω_d / (2π)
        - envelope decay proxy ~ γ
        - autocorrelation at small lags
    """
    x = x.astype(np.float64)
    median = float(np.median(x))
    mad = float(np.median(np.abs(x - median))) + 1e-12
    y = x - median

    std = float(np.std(y))
    p05 = float(np.quantile(y, 0.05))
    p95 = float(np.quantile(y, 0.95))
    iqr = float(p95 - p05)

    # Uniform grid interpolation for FFT
    n_grid = 512
    t0, t1 = float(np.min(t)), float(np.max(t))
    if (t1 - t0) < 1e-6:
        return {
            "mad": mad, "std": std, "iqr": iqr,
            "acf1": 0.0, "acf5": 0.0, "acf15": 0.0,
            "fft_peak_hz": 0.0, "fft_power_ratio": 0.0,
            "env_decay": 0.0, "env_r2": 0.0,
        }

    grid_t = np.linspace(t0, t1, n_grid)
    grid_y = np.interp(grid_t, t, y)
    grid_y = grid_y - np.mean(grid_y)

    # Autocorrelation features
    acf1 = _autocorr(grid_y, 1)
    acf5 = _autocorr(grid_y, 5)
    acf15 = _autocorr(grid_y, 15)

    # FFT: peak frequency in Hz (cycles/sec)
    fft = np.fft.rfft(grid_y)
    power = np.abs(fft) ** 2
    freqs = np.fft.rfftfreq(n_grid, d=(grid_t[1] - grid_t[0]))

    if len(power) > 1:
        power_no_dc = power.copy()
        power_no_dc[0] = 0.0
        idx = int(np.argmax(power_no_dc))
        peak_hz = float(freqs[idx])
        total_power = float(np.sum(power_no_dc) + 1e-12)
        power_ratio = float(power_no_dc[idx] / total_power)
    else:
        peak_hz, power_ratio = 0.0, 0.0

    # Envelope decay estimate:
    # compute absolute signal, take log, fit slope vs time using simple linear regression formula
    # This approximates γ since |A e^{-γt} cos(...)| envelope ~ e^{-γt}.
    abs_y = np.abs(np.interp(grid_t, t, (x - np.median(x))))
    eps = 1e-8
    log_env = np.log(abs_y + eps)

    # Linear regression slope for log_env ~ a + b t, b ~ -γ
    tt = grid_t.reshape(-1)
    yy = log_env.reshape(-1)

    # Focus on upper envelope by taking quantile across sliding windows
    # simple approach: downsample by windows, take 80th percentile as envelope proxy
    w = 16
    env_t = []
    env_y = []
    for i in range(0, len(tt) - w + 1, w):
        window = yy[i:i+w]
        env_t.append(np.mean(tt[i:i+w]))
        env_y.append(np.quantile(window, 0.80))
    env_t = np.array(env_t)
    env_y = np.array(env_y)

    # Fit env_y ~ a + b env_t
    t_mean = float(np.mean(env_t))
    y_mean = float(np.mean(env_y))
    denom = float(np.sum((env_t - t_mean) ** 2) + 1e-12)
    slope = float(np.sum((env_t - t_mean) * (env_y - y_mean)) / denom)
    intercept = y_mean - slope * t_mean

    # R2 for envelope fit
    y_hat = intercept + slope * env_t
    ss_res = float(np.sum((env_y - y_hat) ** 2))
    ss_tot = float(np.sum((env_y - y_mean) ** 2) + 1e-12)
    r2_env = float(1.0 - ss_res / ss_tot)

    env_decay = float(-slope)  # estimate of gamma, roughly

    return {
        "mad": mad,
        "std": std,
        "iqr": iqr,
        "acf1": acf1,
        "acf5": acf5,
        "acf15": acf15,
        "fft_peak_hz": peak_hz,
        "fft_power_ratio": power_ratio,
        "env_decay": env_decay,
        "env_r2": r2_env,
    }


def build_feature_table(series_df: pd.DataFrame) -> pd.DataFrame:
    feats = []
    for system_id, sub in series_df.groupby("system_id", sort=True):
        t = sub["t"].to_numpy()
        x = sub["x"].to_numpy()
        f = extract_features(t, x)
        f["system_id"] = int(system_id)
        feats.append(f)
    return pd.DataFrame(feats).sort_values("system_id").reset_index(drop=True)


# Baseline mechanistic estimators

def baseline_estimate_gamma_omega0(feat_df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    """
    Mechanistic baseline:
        omega_d ≈ 2π * fft_peak_hz
        gamma   ≈ env_decay (from envelope fit)

    Then:
        omega0 ≈ sqrt(omega_d^2 + gamma^2)
    """
    omega_d = 2.0 * np.pi * feat_df["fft_peak_hz"].to_numpy().astype(float)
    gamma_hat = feat_df["env_decay"].to_numpy().astype(float)

    # stabilise
    gamma_hat = np.clip(gamma_hat, 0.0, 5.0)
    omega0_hat = np.sqrt(np.maximum(omega_d**2 + gamma_hat**2, 1e-12))
    return gamma_hat, omega0_hat


# Metrics + uncertainty helpers

def reg_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    return {
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "rmse": float(rmse),
        "r2": float(r2_score(y_true, y_pred)),
    }


def interval_coverage(y_true: np.ndarray, mean: np.ndarray, std: np.ndarray, z: float = 1.96) -> Dict[str, float]:
    """
    Coverage of mean ± z*std interval.
    """
    lo = mean - z * std
    hi = mean + z * std
    covered = (y_true >= lo) & (y_true <= hi)
    return {
        "coverage_95": float(np.mean(covered)),
        "avg_interval_width": float(np.mean(hi - lo)),
    }


def plot_scatter(y_true: np.ndarray, y_pred: np.ndarray, title: str, outpath: str) -> None:
    plt.figure(figsize=(6.4, 5.2))
    plt.scatter(y_true, y_pred, s=18)
    lo = float(min(np.min(y_true), np.min(y_pred)))
    hi = float(max(np.max(y_true), np.max(y_pred)))
    plt.plot([lo, hi], [lo, hi])
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()


def plot_interval_example(t: np.ndarray, x: np.ndarray, title: str, outpath: str) -> None:
    plt.figure(figsize=(6.4, 4.6))
    plt.plot(t, x, linewidth=1.0)
    plt.xlabel("Time (s)")
    plt.ylabel("Displacement x(t)")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outpath, dpi=200)
    plt.close()


# Main experiment

def run_experiment(outdir: str, seed: int = 42, n_systems: int = 400) -> None:
    rng = set_seed(seed)
    ensure_outdir(outdir)

    # 1) Generate dataset
    cfg = OscillatorConfig(n_systems=n_systems)
    series_df, labels_df = simulate_dataset(rng, cfg)

    # Save data previews
    labels_df.to_csv(os.path.join(outdir, "tables", "labels_table.csv"), index=False)
    series_df.head(2500).to_csv(os.path.join(outdir, "tables", "timeseries_preview.csv"), index=False)

    # 2) Feature extraction
    feat_df = build_feature_table(series_df)
    data = feat_df.merge(labels_df, on="system_id", how="inner")

    feature_cols = ["mad", "std", "iqr", "acf1", "acf5", "acf15", "fft_peak_hz", "fft_power_ratio", "env_decay", "env_r2"]
    X = data[feature_cols].to_numpy().astype(float)
    groups = data["system_id"].to_numpy().astype(int)

    y_gamma = data["gamma"].to_numpy().astype(float)
    y_omega0 = data["omega0"].to_numpy().astype(float)

    # 3) Two split strategies
    # Random split
    X_tr_r, X_te_r, yg_tr_r, yg_te_r, y0_tr_r, y0_te_r = train_test_split(
        X, y_gamma, y_omega0, test_size=0.25, random_state=seed
    )

    # Group split
    gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=seed)
    tr_idx, te_idx = next(gss.split(X, y_gamma, groups=groups))
    X_tr_g, X_te_g = X[tr_idx], X[te_idx]
    yg_tr_g, yg_te_g = y_gamma[tr_idx], y_gamma[te_idx]
    y0_tr_g, y0_te_g = y_omega0[tr_idx], y_omega0[te_idx]

    # 4) Baseline mechanistic estimates (no ML)
    gamma_base, omega0_base = baseline_estimate_gamma_omega0(data.loc[te_idx].reset_index(drop=True))

    base_gamma_metrics = reg_metrics(yg_te_g, gamma_base)
    base_omega0_metrics = reg_metrics(y0_te_g, omega0_base)

    # 5) ML models
    # RandomForest (strong non-linear baseline)
    rf_gamma = RandomForestRegressor(n_estimators=700, random_state=seed, min_samples_leaf=3)
    rf_omega0 = RandomForestRegressor(n_estimators=700, random_state=seed, min_samples_leaf=3)

    rf_gamma.fit(X_tr_g, yg_tr_g)
    rf_omega0.fit(X_tr_g, y0_tr_g)

    pred_rf_gamma = rf_gamma.predict(X_te_g)
    pred_rf_omega0 = rf_omega0.predict(X_te_g)

    rf_gamma_metrics = reg_metrics(yg_te_g, pred_rf_gamma)
    rf_omega0_metrics = reg_metrics(y0_te_g, pred_rf_omega0)

    # Gaussian Process Regression (uncertainty-aware)
    # Kernel: constant * RBF + white noise
    kernel = ConstantKernel(1.0, (1e-2, 1e2)) * RBF(length_scale=np.ones(X.shape[1]), length_scale_bounds=(1e-2, 1e2)) \
             + WhiteKernel(noise_level=1e-3, noise_level_bounds=(1e-6, 1e1))

    gpr_gamma = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, random_state=seed)
    gpr_omega0 = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, random_state=seed)

    gpr_gamma.fit(X_tr_g, yg_tr_g)
    gpr_omega0.fit(X_tr_g, y0_tr_g)

    mean_g_gamma, std_g_gamma = gpr_gamma.predict(X_te_g, return_std=True)
    mean_g_omega0, std_g_omega0 = gpr_omega0.predict(X_te_g, return_std=True)

    gpr_gamma_metrics = reg_metrics(yg_te_g, mean_g_gamma)
    gpr_omega0_metrics = reg_metrics(y0_te_g, mean_g_omega0)

    gpr_gamma_unc = interval_coverage(yg_te_g, mean_g_gamma, std_g_gamma)
    gpr_omega0_unc = interval_coverage(y0_te_g, mean_g_omega0, std_g_omega0)

    # 6) OOD test (noise + cadence shift)
    series_ood, labels_ood = simulate_ood_dataset(rng, cfg)
    feat_ood = build_feature_table(series_ood)
    data_ood = feat_ood.merge(labels_ood, on="system_id", how="inner")

    X_ood = data_ood[feature_cols].to_numpy().astype(float)
    yg_ood = data_ood["gamma"].to_numpy().astype(float)
    y0_ood = data_ood["omega0"].to_numpy().astype(float)

    # Evaluate RF and GPR on OOD
    pred_rf_gamma_ood = rf_gamma.predict(X_ood)
    pred_rf_omega0_ood = rf_omega0.predict(X_ood)

    mean_g_gamma_ood, std_g_gamma_ood = gpr_gamma.predict(X_ood, return_std=True)
    mean_g_omega0_ood, std_g_omega0_ood = gpr_omega0.predict(X_ood, return_std=True)

    rf_gamma_ood_metrics = reg_metrics(yg_ood, pred_rf_gamma_ood)
    rf_omega0_ood_metrics = reg_metrics(y0_ood, pred_rf_omega0_ood)

    gpr_gamma_ood_metrics = reg_metrics(yg_ood, mean_g_gamma_ood)
    gpr_omega0_ood_metrics = reg_metrics(y0_ood, mean_g_omega0_ood)

    gpr_gamma_ood_unc = interval_coverage(yg_ood, mean_g_gamma_ood, std_g_gamma_ood)
    gpr_omega0_ood_unc = interval_coverage(y0_ood, mean_g_omega0_ood, std_g_omega0_ood)

    # 7) Save metrics report
    report = {
        "config": vars(cfg),
        "seed": seed,
        "n_systems": n_systems,
        "group_split_results": {
            "baseline_mechanistic": {
                "gamma": base_gamma_metrics,
                "omega0": base_omega0_metrics,
            },
            "random_forest": {
                "gamma": rf_gamma_metrics,
                "omega0": rf_omega0_metrics,
            },
            "gaussian_process": {
                "gamma": {**gpr_gamma_metrics, **gpr_gamma_unc},
                "omega0": {**gpr_omega0_metrics, **gpr_omega0_unc},
            },
        },
        "ood_results": {
            "random_forest": {
                "gamma": rf_gamma_ood_metrics,
                "omega0": rf_omega0_ood_metrics,
            },
            "gaussian_process": {
                "gamma": {**gpr_gamma_ood_metrics, **gpr_gamma_ood_unc},
                "omega0": {**gpr_omega0_ood_metrics, **gpr_omega0_ood_unc},
            },
        },
    }
    save_json(report, os.path.join(outdir, "metrics_report.json"))

    # 8) Produce figures
    figdir = os.path.join(outdir, "figures")

    # Example time series plots (train domain + OOD)
    example_ids = rng.choice(labels_df["system_id"].to_numpy(), size=2, replace=False)
    for sid in example_ids:
        sub = series_df[series_df["system_id"] == sid].sort_values("t")
        plot_interval_example(
            sub["t"].to_numpy(),
            sub["x"].to_numpy(),
            title=f"Damped Oscillator Example (system {sid})",
            outpath=os.path.join(figdir, f"example_series_{sid}.png"),
        )

    example_ids_ood = rng.choice(labels_ood["system_id"].to_numpy(), size=2, replace=False)
    for sid in example_ids_ood:
        sub = series_ood[series_ood["system_id"] == sid].sort_values("t")
        plot_interval_example(
            sub["t"].to_numpy(),
            sub["x"].to_numpy(),
            title=f"OOD Example (system {sid})",
            outpath=os.path.join(figdir, f"example_series_ood_{sid}.png"),
        )

    # Scatter plots (group test)
    plot_scatter(
        yg_te_g, gamma_base,
        title="Baseline Mechanistic: γ (Group Split Test)",
        outpath=os.path.join(figdir, "scatter_baseline_gamma.png"),
    )
    plot_scatter(
        y0_te_g, omega0_base,
        title="Baseline Mechanistic: ω0 (Group Split Test)",
        outpath=os.path.join(figdir, "scatter_baseline_omega0.png"),
    )
    plot_scatter(
        yg_te_g, pred_rf_gamma,
        title="Random Forest: γ (Group Split Test)",
        outpath=os.path.join(figdir, "scatter_rf_gamma.png"),
    )
    plot_scatter(
        y0_te_g, pred_rf_omega0,
        title="Random Forest: ω0 (Group Split Test)",
        outpath=os.path.join(figdir, "scatter_rf_omega0.png"),
    )
    plot_scatter(
        yg_te_g, mean_g_gamma,
        title="Gaussian Process: γ mean (Group Split Test)",
        outpath=os.path.join(figdir, "scatter_gpr_gamma.png"),
    )
    plot_scatter(
        y0_te_g, mean_g_omega0,
        title="Gaussian Process: ω0 mean (Group Split Test)",
        outpath=os.path.join(figdir, "scatter_gpr_omega0.png"),
    )

    # Uncertainty diagnostics: predicted std vs absolute error (GPR)
    abs_err_gamma = np.abs(yg_te_g - mean_g_gamma)
    abs_err_omega0 = np.abs(y0_te_g - mean_g_omega0)

    plt.figure(figsize=(6.4, 5.2))
    plt.scatter(std_g_gamma, abs_err_gamma, s=18)
    plt.xlabel("Predicted std (γ)")
    plt.ylabel("|Error|")
    plt.title("Uncertainty usefulness: GPR std vs |error| (γ)")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "uncertainty_gamma_std_vs_error.png"), dpi=200)
    plt.close()

    plt.figure(figsize=(6.4, 5.2))
    plt.scatter(std_g_omega0, abs_err_omega0, s=18)
    plt.xlabel("Predicted std (ω0)")
    plt.ylabel("|Error|")
    plt.title("Uncertainty usefulness: GPR std vs |error| (ω0)")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "uncertainty_omega0_std_vs_error.png"), dpi=200)
    plt.close()

    # 9) Save compact tables
    # Group split metrics table
    rows = []
    rows.append({"model": "baseline_mechanistic", "target": "gamma", **base_gamma_metrics})
    rows.append({"model": "baseline_mechanistic", "target": "omega0", **base_omega0_metrics})
    rows.append({"model": "random_forest", "target": "gamma", **rf_gamma_metrics})
    rows.append({"model": "random_forest", "target": "omega0", **rf_omega0_metrics})
    rows.append({"model": "gaussian_process", "target": "gamma", **{**gpr_gamma_metrics, **gpr_gamma_unc}})
    rows.append({"model": "gaussian_process", "target": "omega0", **{**gpr_omega0_metrics, **gpr_omega0_unc}})

    group_table = pd.DataFrame(rows)
    group_table.to_csv(os.path.join(outdir, "tables", "group_split_metrics.csv"), index=False)

    # OOD metrics table
    rows_ood = []
    rows_ood.append({"model": "random_forest", "target": "gamma", **rf_gamma_ood_metrics})
    rows_ood.append({"model": "random_forest", "target": "omega0", **rf_omega0_ood_metrics})
    rows_ood.append({"model": "gaussian_process", "target": "gamma", **{**gpr_gamma_ood_metrics, **gpr_gamma_ood_unc}})
    rows_ood.append({"model": "gaussian_process", "target": "omega0", **{**gpr_omega0_ood_metrics, **gpr_omega0_ood_unc}})

    ood_table = pd.DataFrame(rows_ood)
    ood_table.to_csv(os.path.join(outdir, "tables", "ood_metrics.csv"), index=False)

    # 10) Print summary
    print("\n=== Physics/Math Inverse Problem PoC ===")
    print(f"Outputs saved to: {os.path.abspath(outdir)}\n")
    print("Group split test:")
    print("  Baseline mechanistic γ :", base_gamma_metrics)
    print("  Baseline mechanistic ω0:", base_omega0_metrics)
    print("  RF γ :", rf_gamma_metrics)
    print("  RF ω0:", rf_omega0_metrics)
    print("  GPR γ :", {**gpr_gamma_metrics, **gpr_gamma_unc})
    print("  GPR ω0:", {**gpr_omega0_metrics, **gpr_omega0_unc})
    print("\nOOD test:")
    print("  RF γ :", rf_gamma_ood_metrics)
    print("  RF ω0:", rf_omega0_ood_metrics)
    print("  GPR γ :", {**gpr_gamma_ood_metrics, **gpr_gamma_ood_unc})
    print("  GPR ω0:", {**gpr_omega0_ood_metrics, **gpr_omega0_ood_unc})
    print("\nKey figures written to outputs/figures/")
    print("Key tables written to outputs/tables/\n")


def make_argparser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="UH Data Science (Physics/Math) Inverse Problem Proof-of-Concept")
    p.add_argument("--outdir", type=str, default="uh_physics_math_outputs", help="Output directory")
    p.add_argument("--seed", type=int, default=42, help="Random seed")
    p.add_argument("--n_systems", type=int, default=400, help="Number of simulated oscillators")
    return p


def main():
    parser = make_argparser()
    args, _ = parser.parse_known_args()  # Robust in Jupyter + terminal
    run_experiment(outdir=args.outdir, seed=args.seed, n_systems=args.n_systems)


if __name__ == "__main__":
    main()

/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 7 of parameter k1__k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/gaussian_process/kern


=== Physics/Math Inverse Problem PoC ===
Outputs saved to: /Users/petalc01/Data Science Hertfordshire/uh_physics_math_outputs

Group split test:
  Baseline mechanistic γ : {'mae': 0.11148170648767203, 'rmse': 0.17795654283805826, 'r2': 0.6344245828677941}
  Baseline mechanistic ω0: {'mae': 0.3060038658998829, 'rmse': 0.3587402911167564, 'r2': 0.9847316711357893}
  RF γ : {'mae': 0.03470737406859617, 'rmse': 0.055460558201923936, 'r2': 0.9644927092212835}
  RF ω0: {'mae': 0.2926037976288964, 'rmse': 0.36551290850530754, 'r2': 0.9841497312064174}
  GPR γ : {'mae': 0.029246244025369197, 'rmse': 0.0455891117214195, 'r2': 0.9760077308617868, 'coverage_95': 0.93, 'avg_interval_width': 0.14765038874171757}
  GPR ω0: {'mae': 0.26431113598559564, 'rmse': 0.3197315266597091, 'r2': 0.9878716389818153, 'coverage_95': 0.98, 'avg_interval_width': 1.4933269007110834}

OOD test:
  RF γ : {'mae': 0.10577005061123643, 'rmse': 0.16041179107629702, 'r2': 0.7515129011017906}
  RF ω0: {'mae': 0.35601321986

# Gaussian Process Bayesian Optimisation for an Expensive Physics-Style Simulator

## This script demonstrates:
    1) A synthetic but physics-inspired expensive black-box objective:
         - Multi-modal landscape
         - Measurement noise
         - "Experiment cost" penalty
         - Feasible region constraints (like stability limits)
    2) Bayesian optimisation loop:
         - Gaussian Process surrogate (with uncertainty)
         - Acquisition functions: Expected Improvement (EI), UCB
         - Active learning: sequential design under limited evaluation budget
    3) Scientific reporting:
         - convergence curves
         - surrogate predictions + uncertainty
         - acquisition maps (2D)
         - tables + JSON metrics

## Important fixes included:
    Works in Jupyter notebooks (ignores ipykernel "-f" argument)
    Compatible with NumPy 2.x (no np.erf, uses math.erf vectorised)
    Numerical stability for EI (sigma floor)

## Dependencies:
    numpy, pandas, matplotlib, scikit-learn

## Run:
    python uh_bayesopt_physics_poc.py --outdir uh_bo_outputs --acq ei --budget 60

In [23]:
from __future__ import annotations

import os
import json
import math
import argparse
from dataclasses import dataclass
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from sklearn.exceptions import ConvergenceWarning

# Silence ONLY the specific warning you reported (optional but clean).
# Keeping this is harmless even after fixing the kernel bounds.

warnings.filterwarnings(
    "ignore",
    message=r".*parameter k2__noise_level is close to the specified lower bound.*",
    category=ConvergenceWarning,
    module=r"sklearn\.gaussian_process\.kernels",
)

# Repro utilities

def set_seed(seed: int = 42) -> np.random.Generator:
    return np.random.default_rng(seed)


def ensure_outdir(outdir: str) -> str:
    os.makedirs(outdir, exist_ok=True)
    os.makedirs(os.path.join(outdir, "figures"), exist_ok=True)
    os.makedirs(os.path.join(outdir, "tables"), exist_ok=True)
    return outdir


def save_json(obj: Dict, path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)


# Physics-inspired simulator

@dataclass
class SimSettings:
    # domain bounds for two design parameters (Physics/Astro/Math style tuning knobs)
    # x1: e.g. driving amplitude
    # x2: e.g. coupling strength
    x1_bounds: Tuple[float, float] = (-2.0, 2.0)
    x2_bounds: Tuple[float, float] = (-2.0, 2.0)

    # noise level representing stochastic experiment measurement noise
    noise_sigma: float = 0.08

    # cost penalty strength
    cost_lambda: float = 0.15

    # feasibility (stability) constraint severity
    constraint_strength: float = 8.0


def simulator_objective(
    rng: np.random.Generator,
    X: np.ndarray,
    cfg: SimSettings,
    noisy: bool = True,
    shift: float = 0.0,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Expensive black-box objective:
        score = "physics performance" - cost_lambda * "experiment cost" - constraint penalty + noise

    Physics performance:
        mixture of oscillatory + smooth peaks (multi-modal)
    Cost:
        encourages cheaper experiments (like resource usage or time)
    Constraint:
        feasible region in a curved manifold (simulating stability)
    shift:
        adds a distribution shift to simulate lab drift or regime change
    """
    x1 = X[:, 0]
    x2 = X[:, 1]

    # 1) "Physics performance" term (multi-modal, smooth + oscillatory)
    # Think: resonance + interference patterns
    perf = (
        1.7 * np.sin(2.6 * (x1 + shift)) * np.cos(2.2 * (x2 - 0.3 * shift))
        + 0.9 * np.exp(-((x1 - 0.8 + 0.2 * shift) ** 2 + (x2 + 0.6) ** 2) / 0.35)
        + 1.1 * np.exp(-((x1 + 1.1) ** 2 + (x2 - 0.9 + 0.1 * shift) ** 2) / 0.25)
        - 0.15 * (x1**2 + x2**2)
    )

    # 2) Cost term (penalise "expensive" settings)
    # Example: high |x1| increases power usage, high x2 increases complexity
    cost = 0.6 * (np.abs(x1) ** 1.3) + 0.4 * (np.abs(x2) ** 1.6)

    # 3) Feasibility constraint: stable if inside a curved boundary
    # Example constraint: x2 <= 1.2 - 0.5*x1^2 (curved stability envelope)
    stability_limit = 1.2 - 0.5 * (x1**2)
    violation = np.maximum(0.0, x2 - stability_limit)
    penalty = cfg.constraint_strength * (violation ** 2)

    score_clean = perf - cfg.cost_lambda * cost - penalty

    # Noisy measurement
    if noisy:
        eps = rng.normal(0.0, cfg.noise_sigma, size=score_clean.shape)
        score = score_clean + eps
    else:
        score = score_clean

    return score.astype(float), cost.astype(float)


def latin_hypercube_2d(
    rng: np.random.Generator,
    n: int,
    bounds: Tuple[Tuple[float, float], Tuple[float, float]],
) -> np.ndarray:
    """
    Simple Latin Hypercube Sampling in 2D to seed BO.
    """
    (a1, b1), (a2, b2) = bounds
    u1 = (np.arange(n) + rng.random(n)) / n
    u2 = (np.arange(n) + rng.random(n)) / n
    rng.shuffle(u1)
    rng.shuffle(u2)
    x1 = a1 + (b1 - a1) * u1
    x2 = a2 + (b2 - a2) * u2
    return np.vstack([x1, x2]).T


# Gaussian Process surrogate

def make_gp(seed: int = 42) -> GaussianProcessRegressor:
    kernel = (
        ConstantKernel(1.0, (1e-2, 1e2))
        * RBF(length_scale=np.ones(2), length_scale_bounds=(1e-2, 1e2))
        + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-6, 1e1))
    )
    gp = GaussianProcessRegressor(
        kernel=kernel,
        normalize_y=True,
        random_state=seed,
        n_restarts_optimizer=4,
    )
    return gp


# Acquisition functions

def normal_pdf(z: np.ndarray) -> np.ndarray:
    return (1.0 / np.sqrt(2.0 * np.pi)) * np.exp(-0.5 * z**2)


def normal_cdf(z: np.ndarray) -> np.ndarray:
    """
    CDF of standard normal.
    NumPy 2.x removed np.erf, so we use math.erf and vectorise it.
    """
    erf_vec = np.vectorize(math.erf)
    return 0.5 * (1.0 + erf_vec(z / np.sqrt(2.0)))


def expected_improvement(mu: np.ndarray, sigma: np.ndarray, best_y: float, xi: float = 0.01) -> np.ndarray:
    """
    EI for maximisation.
    """
    sigma = np.maximum(sigma, 1e-12)  # numerical stability
    imp = mu - best_y - xi
    z = imp / sigma
    ei = imp * normal_cdf(z) + sigma * normal_pdf(z)
    return np.maximum(ei, 0.0)


def upper_confidence_bound(mu: np.ndarray, sigma: np.ndarray, beta: float = 2.0) -> np.ndarray:
    """
    UCB for maximisation.
    """
    return mu + beta * sigma


# BO loop

def propose_next_point(
    rng: np.random.Generator,
    gp: GaussianProcessRegressor,
    bounds: Tuple[Tuple[float, float], Tuple[float, float]],
    acq: str,
    y_best: float,
    n_candidates: int = 6000,
) -> np.ndarray:
    """
    Use random candidate search over acquisition function for simplicity and clarity.
    """
    (a1, b1), (a2, b2) = bounds
    Xcand = np.column_stack([
        rng.uniform(a1, b1, size=n_candidates),
        rng.uniform(a2, b2, size=n_candidates),
    ])

    mu, std = gp.predict(Xcand, return_std=True)

    if acq.lower() == "ei":
        values = expected_improvement(mu, std, best_y=y_best, xi=0.01)
    elif acq.lower() == "ucb":
        values = upper_confidence_bound(mu, std, beta=2.2)
    else:
        raise ValueError("acq must be one of: ei, ucb")

    idx = int(np.argmax(values))
    return Xcand[idx:idx + 1, :]


def run_bayesopt(
    outdir: str,
    seed: int = 42,
    budget: int = 60,
    init_points: int = 12,
    acq: str = "ei",
    n_candidates: int = 6000,
    shift_test: bool = True,
) -> None:
    rng = set_seed(seed)
    ensure_outdir(outdir)

    cfg = SimSettings()
    bounds = (cfg.x1_bounds, cfg.x2_bounds)

    # 1) Initialise design with LHS
    X = latin_hypercube_2d(rng, init_points, bounds)
    y, cost = simulator_objective(rng, X, cfg, noisy=True, shift=0.0)

    trace_rows = []
    for i in range(init_points):
        trace_rows.append({
            "iter": i,
            "x1": float(X[i, 0]),
            "x2": float(X[i, 1]),
            "y": float(y[i]),
            "cost": float(cost[i]),
            "phase": "init",
        })

    # 2) BO loop
    gp = make_gp(seed=seed)

    for it in range(init_points, budget):
        gp.fit(X, y)

        y_best = float(np.max(y))

        # Propose next
        x_next = propose_next_point(
            rng=rng,
            gp=gp,
            bounds=bounds,
            acq=acq,
            y_best=y_best,
            n_candidates=n_candidates,
        )

        # Evaluate simulator
        y_next, cost_next = simulator_objective(rng, x_next, cfg, noisy=True, shift=0.0)

        # Append
        X = np.vstack([X, x_next])
        y = np.concatenate([y, y_next])
        cost = np.concatenate([cost, cost_next])

        trace_rows.append({
            "iter": it,
            "x1": float(x_next[0, 0]),
            "x2": float(x_next[0, 1]),
            "y": float(y_next[0]),
            "cost": float(cost_next[0]),
            "phase": "bo",
            "best_y_before": y_best,
        })

    # Final fit
    gp.fit(X, y)
    y_best = float(np.max(y))
    best_idx = int(np.argmax(y))
    x_best = X[best_idx]

    # 3) Optional OOD shift evaluation
    shift_report = None
    if shift_test:
        x_test = x_best.reshape(1, 2)
        y_in, _ = simulator_objective(rng, x_test, cfg, noisy=False, shift=0.0)
        y_shifted, _ = simulator_objective(rng, x_test, cfg, noisy=False, shift=0.35)
        shift_report = {
            "best_x": {"x1": float(x_best[0]), "x2": float(x_best[1])},
            "score_in_domain_noiseless": float(y_in[0]),
            "score_shifted_noiseless": float(y_shifted[0]),
            "drop_due_to_shift": float(y_in[0] - y_shifted[0]),
        }

    # 4) Save tables
    trace_df = pd.DataFrame(trace_rows)
    trace_df.to_csv(os.path.join(outdir, "tables", "bo_trace.csv"), index=False)

    # 5) Produce scientific plots
    figdir = os.path.join(outdir, "figures")

    # Convergence curve
    best_curve = np.maximum.accumulate(trace_df["y"].to_numpy())
    plt.figure(figsize=(7.0, 4.8))
    plt.plot(trace_df["iter"], best_curve, marker="o", linewidth=1.2)
    plt.xlabel("Iteration")
    plt.ylabel("Best observed objective")
    plt.title(f"Bayesian Optimisation Convergence ({acq.upper()})")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "convergence.png"), dpi=200)
    plt.close()

    # 2D surrogate map (mean and std)
    grid_n = 140
    x1g = np.linspace(cfg.x1_bounds[0], cfg.x1_bounds[1], grid_n)
    x2g = np.linspace(cfg.x2_bounds[0], cfg.x2_bounds[1], grid_n)
    xx1, xx2 = np.meshgrid(x1g, x2g)
    Xgrid = np.column_stack([xx1.ravel(), xx2.ravel()])

    mu, std = gp.predict(Xgrid, return_std=True)
    MU = mu.reshape(grid_n, grid_n)
    STD = std.reshape(grid_n, grid_n)

    # Training points overlay
    plt.figure(figsize=(7.0, 5.6))
    plt.imshow(
        MU,
        origin="lower",
        aspect="auto",
        extent=[cfg.x1_bounds[0], cfg.x1_bounds[1], cfg.x2_bounds[0], cfg.x2_bounds[1]],
    )
    plt.scatter(X[:, 0], X[:, 1], s=18)
    plt.scatter(x_best[0], x_best[1], s=80, marker="x")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title("GP Surrogate Mean (sampled points + best point)")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "gp_mean_map.png"), dpi=200)
    plt.close()

    plt.figure(figsize=(7.0, 5.6))
    plt.imshow(
        STD,
        origin="lower",
        aspect="auto",
        extent=[cfg.x1_bounds[0], cfg.x1_bounds[1], cfg.x2_bounds[0], cfg.x2_bounds[1]],
    )
    plt.scatter(X[:, 0], X[:, 1], s=18)
    plt.scatter(x_best[0], x_best[1], s=80, marker="x")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title("GP Surrogate Uncertainty (std)")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "gp_std_map.png"), dpi=200)
    plt.close()

    # Acquisition map
    if acq.lower() == "ei":
        acq_vals = expected_improvement(mu, std, best_y=float(np.max(y)), xi=0.01)
    else:
        acq_vals = upper_confidence_bound(mu, std, beta=2.2)
    ACQ = acq_vals.reshape(grid_n, grid_n)

    plt.figure(figsize=(7.0, 5.6))
    plt.imshow(
        ACQ,
        origin="lower",
        aspect="auto",
        extent=[cfg.x1_bounds[0], cfg.x1_bounds[1], cfg.x2_bounds[0], cfg.x2_bounds[1]],
    )
    plt.scatter(X[:, 0], X[:, 1], s=18)
    plt.scatter(x_best[0], x_best[1], s=80, marker="x")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title(f"Acquisition Map ({acq.upper()})")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "acquisition_map.png"), dpi=200)
    plt.close()

    # Compare BO vs random search baseline (fair budget)
    Xrand = np.column_stack([
        rng.uniform(cfg.x1_bounds[0], cfg.x1_bounds[1], size=budget),
        rng.uniform(cfg.x2_bounds[0], cfg.x2_bounds[1], size=budget),
    ])
    yrand, _ = simulator_objective(rng, Xrand, cfg, noisy=True, shift=0.0)
    best_rand = np.maximum.accumulate(yrand)

    plt.figure(figsize=(7.0, 4.8))
    plt.plot(np.arange(budget), best_curve[:budget], marker="o", linewidth=1.2, label=f"BO ({acq.upper()})")
    plt.plot(np.arange(budget), best_rand, marker="o", linewidth=1.2, label="Random search")
    plt.xlabel("Iteration")
    plt.ylabel("Best observed objective")
    plt.title("Sample Efficiency: Bayesian Optimisation vs Random")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "bo_vs_random.png"), dpi=200)
    plt.close()

    # 6) Save metrics report
    metrics = {
        "seed": seed,
        "budget": budget,
        "init_points": init_points,
        "acquisition": acq,
        "best_y_observed": float(y_best),
        "best_x_observed": {"x1": float(x_best[0]), "x2": float(x_best[1])},
        "random_baseline_best_y": float(np.max(yrand)),
        "efficiency_gap_best_y": float(y_best - np.max(yrand)),
        "gp_kernel": str(gp.kernel_),
        "ood_shift_check": shift_report,
    }
    save_json(metrics, os.path.join(outdir, "metrics_report.json"))

    # 7) Print concise summary
    print("\n=== GP Bayesian Optimisation PoC ===")
    print(f"Outputs saved to: {os.path.abspath(outdir)}\n")
    print("Best found design:")
    print("  x1 =", float(x_best[0]))
    print("  x2 =", float(x_best[1]))
    print("  best y =", float(y_best))
    print("\nRandom search best y (same budget):", float(np.max(yrand)))
    print("Efficiency gap (BO - random):", float(y_best - np.max(yrand)))

    if shift_report is not None:
        print("\nOOD shift check at best point (noiseless):")
        print("  in-domain score :", shift_report["score_in_domain_noiseless"])
        print("  shifted score   :", shift_report["score_shifted_noiseless"])
        print("  drop            :", shift_report["drop_due_to_shift"])

    print("\nKey figures written to outputs/figures/")
    print("Key tables written to outputs/tables/\n")


def make_argparser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="UH Data Science PoC: Gaussian Process Bayesian Optimisation")
    p.add_argument("--outdir", type=str, default="uh_bo_outputs", help="Output directory")
    p.add_argument("--seed", type=int, default=42, help="Random seed")
    p.add_argument("--budget", type=int, default=60, help="Total number of evaluations (including init)")
    p.add_argument("--init_points", type=int, default=12, help="Initial points before BO loop")
    p.add_argument("--acq", type=str, default="ei", choices=["ei", "ucb"], help="Acquisition function")
    p.add_argument("--n_candidates", type=int, default=6000, help="Random candidates for acquisition maximisation")
    p.add_argument("--no_shift_test", action="store_true", help="Disable OOD shift check")
    return p


def main() -> None:
    parser = make_argparser()
    args, _ = parser.parse_known_args()  # Robust in Jupyter + terminal (ignores -f kernel.json)
    run_bayesopt(
        outdir=args.outdir,
        seed=args.seed,
        budget=args.budget,
        init_points=args.init_points,
        acq=args.acq,
        n_candidates=args.n_candidates,
        shift_test=(not args.no_shift_test),
    )


if __name__ == "__main__":
    main()


=== GP Bayesian Optimisation PoC ===
Outputs saved to: /Users/petalc01/Data Science Hertfordshire/uh_bo_outputs

Best found design:
  x1 = 0.5748359092858863
  x2 = -0.11639897192768656
  best y = 2.119765312067755

Random search best y (same budget): 1.6852151086269096
Efficiency gap (BO - random): 0.4345502034408453

OOD shift check at best point (noiseless):
  in-domain score : 1.9416040241658161
  shifted score   : 1.343030715628198
  drop            : 0.5985733085376181

Key figures written to outputs/figures/
Key tables written to outputs/tables/



# Multi-Objective Bayesian Optimisation (Pareto Front + Hypervolume Tracking) for a Physics-Style Expensive Simulator

## What this script demonstrates:
    1) Multi-objective optimisation:
         - Objective 1: maximise "physics performance"
         - Objective 2: minimise "experiment cost"
       (trade-off is unavoidable and realistic)
    2) Bayesian optimisation loop:
         - GP surrogate models for each objective
         - Scalarisation sampling to explore Pareto front
         - Exploration bonus using GP uncertainty
    3) Pareto front estimation:
         - non-dominated sorting
         - hypervolume metric improvement over time
    4) Scientific reporting:
         - Pareto plots
         - hypervolume curve
         - trace tables + JSON metrics

## Dependencies:
    numpy, pandas, matplotlib, scikit-learn

## Run:
    python uh_mobo_pareto_physics_poc.py --outdir uh_mobo_outputs --budget 80

In [30]:
from __future__ import annotations

import os
import json
import argparse
from dataclasses import dataclass
from typing import Dict, Tuple, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from sklearn.exceptions import ConvergenceWarning

# Silence ONLY the specific warning you reported (optional but clean).
# Keeping this is harmless even after fixing the kernel bounds.

warnings.filterwarnings(
    "ignore",
    message=r".*parameter k2__noise_level is close to the specified lower bound.*",
    category=ConvergenceWarning,
    module=r"sklearn\.gaussian_process\.kernels",
)


# Repro utilities

def set_seed(seed: int = 42) -> np.random.Generator:
    return np.random.default_rng(seed)


def ensure_outdir(outdir: str) -> str:
    os.makedirs(outdir, exist_ok=True)
    os.makedirs(os.path.join(outdir, "figures"), exist_ok=True)
    os.makedirs(os.path.join(outdir, "tables"), exist_ok=True)
    return outdir


def save_json(obj: Dict, path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)


# Physics-inspired multi-objective simulator

@dataclass
class SimSettings:
    x1_bounds: Tuple[float, float] = (-2.0, 2.0)
    x2_bounds: Tuple[float, float] = (-2.0, 2.0)
    noise_sigma: float = 0.08
    constraint_strength: float = 7.5


def simulator_multiobjective(
    rng: np.random.Generator,
    X: np.ndarray,
    cfg: SimSettings,
    noisy: bool = True,
    shift: float = 0.0,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Returns:
        f1: performance score to MAXIMISE
        f2: experiment cost to MINIMISE
    """
    x1 = X[:, 0]
    x2 = X[:, 1]

    # Physics performance (multi-modal)
    perf = (
        1.8 * np.sin(2.6 * (x1 + shift)) * np.cos(2.2 * (x2 - 0.25 * shift))
        + 0.95 * np.exp(-((x1 - 0.9 + 0.2 * shift) ** 2 + (x2 + 0.55) ** 2) / 0.35)
        + 1.10 * np.exp(-((x1 + 1.05) ** 2 + (x2 - 0.95 + 0.1 * shift) ** 2) / 0.25)
        - 0.12 * (x1**2 + x2**2)
    )

    # Experiment cost (positive)
    cost = 0.55 * (np.abs(x1) ** 1.3) + 0.45 * (np.abs(x2) ** 1.6)

    # Feasibility/stability penalty (subtract from performance)
    stability_limit = 1.2 - 0.5 * (x1**2)
    violation = np.maximum(0.0, x2 - stability_limit)
    penalty = cfg.constraint_strength * (violation ** 2)

    f1_clean = perf - penalty
    f2_clean = cost

    if noisy:
        f1 = f1_clean + rng.normal(0.0, cfg.noise_sigma, size=f1_clean.shape)
        f2 = f2_clean + rng.normal(0.0, 0.02, size=f2_clean.shape)  # small cost noise
    else:
        f1, f2 = f1_clean, f2_clean

    return f1.astype(float), f2.astype(float)


def latin_hypercube_2d(rng: np.random.Generator, n: int, bounds: Tuple[Tuple[float, float], Tuple[float, float]]) -> np.ndarray:
    (a1, b1), (a2, b2) = bounds
    u1 = (np.arange(n) + rng.random(n)) / n
    u2 = (np.arange(n) + rng.random(n)) / n
    rng.shuffle(u1)
    rng.shuffle(u2)
    x1 = a1 + (b1 - a1) * u1
    x2 = a2 + (b2 - a2) * u2
    return np.vstack([x1, x2]).T


# Pareto and hypervolume

def pareto_mask_maxmin(f1: np.ndarray, f2: np.ndarray) -> np.ndarray:
    """
    Non-dominated points where:
        f1: maximise
        f2: minimise
    A point i is dominated if there exists j with:
        f1_j >= f1_i and f2_j <= f2_i, with at least one strict.
    """
    n = len(f1)
    dominated = np.zeros(n, dtype=bool)

    for i in range(n):
        if dominated[i]:
            continue
        for j in range(n):
            if i == j:
                continue
            if (f1[j] >= f1[i] and f2[j] <= f2[i]) and (f1[j] > f1[i] or f2[j] < f2[i]):
                dominated[i] = True
                break

    return ~dominated


def hypervolume_2d_maxmin(f1: np.ndarray, f2: np.ndarray, ref: Tuple[float, float]) -> float:
    """
    Hypervolume for 2D with:
        maximise f1, minimise f2
    Reference point ref = (f1_ref, f2_ref) should be "worse" than all points:
        f1_ref small, f2_ref large.
    We compute area dominated by Pareto front to the reference.

    Implementation:
        Sort Pareto points by f2 ascending (lower cost better),
        accumulate rectangles in (f1, f2) plane.
    """
    mask = pareto_mask_maxmin(f1, f2)
    p1 = f1[mask]
    p2 = f2[mask]

    if len(p1) == 0:
        return 0.0

    # sort by cost increasing
    order = np.argsort(p2)
    p1 = p1[order]
    p2 = p2[order]

    f1_ref, f2_ref = ref
    hv = 0.0
    best_f1_so_far = f1_ref

    # Walk along increasing cost, add slices where f1 improves
    for i in range(len(p1)):
        # width in f1 direction: improvement over best_f1_so_far
        width = max(0.0, p1[i] - best_f1_so_far)
        # height in f2 direction: from this cost to reference cost (since we minimise f2)
        height = max(0.0, f2_ref - p2[i])
        hv += width * height
        best_f1_so_far = max(best_f1_so_far, p1[i])

    return float(hv)


# GP models

def make_gp(seed: int = 42) -> GaussianProcessRegressor:
    kernel = (
        ConstantKernel(1.0, (1e-2, 1e2))
        * RBF(length_scale=np.ones(2), length_scale_bounds=(1e-2, 1e2))
        + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-6, 1e1))
    )
    return GaussianProcessRegressor(
        kernel=kernel,
        normalize_y=True,
        random_state=seed,
        n_restarts_optimizer=4,
    )


# MOBO acquisition (scalarisation + uncertainty bonus)

def propose_next_point_mobo(
    rng: np.random.Generator,
    gp1: GaussianProcessRegressor,
    gp2: GaussianProcessRegressor,
    bounds: Tuple[Tuple[float, float], Tuple[float, float]],
    n_candidates: int = 8000,
    explore_bonus: float = 0.15,
) -> Tuple[np.ndarray, Dict]:
    """
    Multi-objective acquisition via random scalarisation.

    We sample a weight w in (0,1):
        score = w * (standardised f1) - (1-w) * (standardised f2) + explore_bonus * uncertainty

    Since f1 is maximised and f2 minimised, we subtract f2 component.
    """
    (a1, b1), (a2, b2) = bounds
    Xcand = np.column_stack([
        rng.uniform(a1, b1, size=n_candidates),
        rng.uniform(a2, b2, size=n_candidates),
    ])

    mu1, std1 = gp1.predict(Xcand, return_std=True)
    mu2, std2 = gp2.predict(Xcand, return_std=True)

    # Standardise predicted means to comparable scales
    m1 = (mu1 - np.mean(mu1)) / (np.std(mu1) + 1e-12)
    m2 = (mu2 - np.mean(mu2)) / (np.std(mu2) + 1e-12)

    # exploration proxy: combine uncertainties
    unc = (std1 / (np.mean(std1) + 1e-12)) + (std2 / (np.mean(std2) + 1e-12))

    # random scalarisation weight
    w = float(rng.uniform(0.15, 0.85))
    score = w * m1 - (1.0 - w) * m2 + explore_bonus * unc

    idx = int(np.argmax(score))
    meta = {"w": w, "explore_bonus": explore_bonus}
    return Xcand[idx:idx+1, :], meta


# Experiment

def run_mobo(
    outdir: str,
    seed: int = 42,
    budget: int = 80,
    init_points: int = 14,
    n_candidates: int = 8000,
    explore_bonus: float = 0.15,
) -> None:
    rng = set_seed(seed)
    ensure_outdir(outdir)

    cfg = SimSettings()
    bounds = (cfg.x1_bounds, cfg.x2_bounds)

    # Initialise with LHS
    X = latin_hypercube_2d(rng, init_points, bounds)
    f1, f2 = simulator_multiobjective(rng, X, cfg, noisy=True, shift=0.0)

    trace = []
    for i in range(init_points):
        trace.append({
            "iter": i,
            "x1": float(X[i, 0]),
            "x2": float(X[i, 1]),
            "f1_perf": float(f1[i]),
            "f2_cost": float(f2[i]),
            "phase": "init",
        })

    # MOBO loop
    gp1 = make_gp(seed=seed)
    gp2 = make_gp(seed=seed)

    # choose reference point for hypervolume (worse than likely observed)
    # we can set:
    #   f1_ref: slightly below min observed
    #   f2_ref: slightly above max observed
    f1_ref = float(np.min(f1) - 0.5)
    f2_ref = float(np.max(f2) + 0.5)
    hv_curve = []

    for it in range(init_points, budget):
        gp1.fit(X, f1)
        gp2.fit(X, f2)

        # hypervolume progress
        hv = hypervolume_2d_maxmin(f1, f2, ref=(f1_ref, f2_ref))
        hv_curve.append(hv)

        x_next, meta = propose_next_point_mobo(
            rng=rng,
            gp1=gp1,
            gp2=gp2,
            bounds=bounds,
            n_candidates=n_candidates,
            explore_bonus=explore_bonus,
        )

        f1_next, f2_next = simulator_multiobjective(rng, x_next, cfg, noisy=True, shift=0.0)

        X = np.vstack([X, x_next])
        f1 = np.concatenate([f1, f1_next])
        f2 = np.concatenate([f2, f2_next])

        trace.append({
            "iter": it,
            "x1": float(x_next[0, 0]),
            "x2": float(x_next[0, 1]),
            "f1_perf": float(f1_next[0]),
            "f2_cost": float(f2_next[0]),
            "phase": "mobo",
            "scalar_weight_w": meta["w"],
            "explore_bonus": meta["explore_bonus"],
            "hypervolume_before": hv,
        })

    # final HV
    hv_final = hypervolume_2d_maxmin(f1, f2, ref=(f1_ref, f2_ref))
    hv_curve.append(hv_final)

    # Save trace
    trace_df = pd.DataFrame(trace)
    trace_df.to_csv(os.path.join(outdir, "tables", "mobo_trace.csv"), index=False)

    # Pareto front
    mask = pareto_mask_maxmin(f1, f2)
    pareto_df = pd.DataFrame({
        "x1": X[mask, 0],
        "x2": X[mask, 1],
        "f1_perf": f1[mask],
        "f2_cost": f2[mask],
    }).sort_values("f2_cost").reset_index(drop=True)

    pareto_df.to_csv(os.path.join(outdir, "tables", "pareto_front.csv"), index=False)

    # Figures
    figdir = os.path.join(outdir, "figures")

    # Pareto scatter
    plt.figure(figsize=(7.0, 5.2))
    plt.scatter(f2, f1, s=18, label="All evaluated")
    plt.scatter(pareto_df["f2_cost"], pareto_df["f1_perf"], s=35, label="Pareto front")
    plt.xlabel("Cost (minimise)")
    plt.ylabel("Performance (maximise)")
    plt.title("Pareto Front Discovery (Multi-objective Optimisation)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "pareto_front.png"), dpi=200)
    plt.close()

    # Hypervolume curve
    plt.figure(figsize=(7.0, 4.8))
    x_it = np.arange(init_points, budget + 1)
    plt.plot(x_it, hv_curve, marker="o", linewidth=1.2)
    plt.xlabel("Iteration")
    plt.ylabel("Hypervolume (higher is better)")
    plt.title("Hypervolume Improvement over Iterations")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "hypervolume_curve.png"), dpi=200)
    plt.close()

    # Pareto in design space (x1,x2)
    plt.figure(figsize=(7.0, 5.2))
    plt.scatter(X[:, 0], X[:, 1], s=18, label="All evaluated")
    plt.scatter(pareto_df["x1"], pareto_df["x2"], s=40, label="Pareto designs")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title("Design Space, evaluated points and Pareto designs")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "design_space_pareto.png"), dpi=200)
    plt.close()

    # Save metrics
    metrics = {
        "seed": seed,
        "budget": budget,
        "init_points": init_points,
        "n_candidates": n_candidates,
        "explore_bonus": explore_bonus,
        "hypervolume_ref_point": {"f1_ref": f1_ref, "f2_ref": f2_ref},
        "hypervolume_final": float(hv_final),
        "n_pareto_points": int(len(pareto_df)),
        "best_perf_overall": float(np.max(f1)),
        "min_cost_overall": float(np.min(f2)),
    }
    save_json(metrics, os.path.join(outdir, "metrics_report.json"))

    # Print summary
    print("\n=== Multi-objective Bayesian Optimisation PoC ===")
    print(f"Outputs saved to: {os.path.abspath(outdir)}\n")
    print("Pareto front size:", len(pareto_df))
    print("Final hypervolume:", float(hv_final))
    print("Best performance observed:", float(np.max(f1)))
    print("Minimum cost observed:", float(np.min(f2)))
    print("\nKey figures written to outputs/figures/")
    print("Key tables written to outputs/tables/\n")


def make_argparser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="UH MOBO PoC: Pareto front + hypervolume tracking")
    p.add_argument("--outdir", type=str, default="uh_mobo_outputs", help="Output directory")
    p.add_argument("--seed", type=int, default=42, help="Random seed")
    p.add_argument("--budget", type=int, default=80, help="Total evaluations (including init)")
    p.add_argument("--init_points", type=int, default=14, help="Initial LHS points")
    p.add_argument("--n_candidates", type=int, default=8000, help="Random candidates per iteration")
    p.add_argument("--explore_bonus", type=float, default=0.15, help="Exploration strength")
    return p


def main() -> None:
    parser = make_argparser()
    args, _ = parser.parse_known_args()  # Robust in Jupyter + terminal (ignores -f kernel.json)
    run_mobo(
        outdir=args.outdir,
        seed=args.seed,
        budget=args.budget,
        init_points=args.init_points,
        n_candidates=args.n_candidates,
        explore_bonus=args.explore_bonus,
    )


if __name__ == "__main__":
    main()


=== Multi-objective Bayesian Optimisation PoC ===
Outputs saved to: /Users/petalc01/Data Science Hertfordshire/uh_mobo_outputs

Pareto front size: 12
Final hypervolume: 132.9772940947282
Best performance observed: 1.8976758678346248
Minimum cost observed: -0.020578763202621177

Key figures written to outputs/figures/
Key tables written to outputs/tables/



# Multi-Objective Bayesian Optimisation (MOBO++) for Physics-Style Simulator: Pareto Front + Pareto Ranks + Hypervolume Contribution + Epsilon-Constraint + Batch BO

## Objectives:
    f1: maximise performance
    f2: minimise cost

## What this script demonstrates:
    1) A realistic multi-objective scientific optimisation problem (performance vs cost).
    2) GP surrogates + uncertainty, active learning.
    3) Batch BO (K proposals per iteration) for realistic experimental throughput.
    4) Pareto ranks (non-dominated sorting levels).
    5) Hypervolume tracking over time.
    6) Hypervolume contribution of Pareto points (impact of each design).
    7) Epsilon-constraint extraction: best performance under specified cost thresholds.
    8) Reproducible artefacts: figures, tables, JSON metrics.

## Dependencies:
    numpy, pandas, matplotlib, scikit-learn

## Run:
    python uh_mobo_pareto_physics_poc_plus.py --outdir uh_mobo_plus --budget 100 --batch_size 4


In [33]:
from __future__ import annotations

import os
import json
import argparse
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel
from sklearn.exceptions import ConvergenceWarning

# Silence ONLY the specific warning you reported (optional but clean).
# Keeping this is harmless even after fixing the kernel bounds.

warnings.filterwarnings(
    "ignore",
    message=r".*parameter k2__noise_level is close to the specified lower bound.*",
    category=ConvergenceWarning,
    module=r"sklearn\.gaussian_process\.kernels",
)


# Repro utilities


def set_seed(seed: int = 42) -> np.random.Generator:
    return np.random.default_rng(seed)


def ensure_outdir(outdir: str) -> str:
    os.makedirs(outdir, exist_ok=True)
    os.makedirs(os.path.join(outdir, "figures"), exist_ok=True)
    os.makedirs(os.path.join(outdir, "tables"), exist_ok=True)
    return outdir


def save_json(obj: Dict, path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)


# Physics-inspired simulator


@dataclass
class SimSettings:
    x1_bounds: Tuple[float, float] = (-2.0, 2.0)
    x2_bounds: Tuple[float, float] = (-2.0, 2.0)
    noise_sigma: float = 0.08
    constraint_strength: float = 7.5


def simulator_multiobjective(
    rng: np.random.Generator,
    X: np.ndarray,
    cfg: SimSettings,
    noisy: bool = True,
    shift: float = 0.0,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Returns:
        f1: performance score to MAXIMISE
        f2: experiment cost to MINIMISE
    """
    x1 = X[:, 0]
    x2 = X[:, 1]

    # Performance landscape: resonance/interference + multiple peaks
    perf = (
        1.8 * np.sin(2.6 * (x1 + shift)) * np.cos(2.2 * (x2 - 0.25 * shift))
        + 0.95 * np.exp(-((x1 - 0.9 + 0.2 * shift) ** 2 + (x2 + 0.55) ** 2) / 0.35)
        + 1.10 * np.exp(-((x1 + 1.05) ** 2 + (x2 - 0.95 + 0.1 * shift) ** 2) / 0.25)
        - 0.12 * (x1**2 + x2**2)
    )

    # Cost is always positive, grows with magnitude
    cost = 0.55 * (np.abs(x1) ** 1.3) + 0.45 * (np.abs(x2) ** 1.6)

    # Soft stability constraint, penalise infeasible x2 above a curved boundary
    stability_limit = 1.2 - 0.5 * (x1**2)
    violation = np.maximum(0.0, x2 - stability_limit)
    penalty = cfg.constraint_strength * (violation ** 2)

    f1_clean = perf - penalty
    f2_clean = cost

    if noisy:
        f1 = f1_clean + rng.normal(0.0, cfg.noise_sigma, size=f1_clean.shape)
        f2 = f2_clean + rng.normal(0.0, 0.02, size=f2_clean.shape)
    else:
        f1, f2 = f1_clean, f2_clean

    return f1.astype(float), f2.astype(float)


def latin_hypercube_2d(
    rng: np.random.Generator,
    n: int,
    bounds: Tuple[Tuple[float, float], Tuple[float, float]]
) -> np.ndarray:
    (a1, b1), (a2, b2) = bounds
    u1 = (np.arange(n) + rng.random(n)) / n
    u2 = (np.arange(n) + rng.random(n)) / n
    rng.shuffle(u1)
    rng.shuffle(u2)
    x1 = a1 + (b1 - a1) * u1
    x2 = a2 + (b2 - a2) * u2
    return np.vstack([x1, x2]).T


# Pareto tools


def dominates_maxmin(a_f1: float, a_f2: float, b_f1: float, b_f2: float) -> bool:
    """
    True if A dominates B for:
        maximise f1
        minimise f2
    """
    return (a_f1 >= b_f1 and a_f2 <= b_f2) and (a_f1 > b_f1 or a_f2 < b_f2)


def pareto_mask_maxmin(f1: np.ndarray, f2: np.ndarray) -> np.ndarray:
    n = len(f1)
    dominated = np.zeros(n, dtype=bool)
    for i in range(n):
        if dominated[i]:
            continue
        for j in range(n):
            if i == j:
                continue
            if dominates_maxmin(f1[j], f2[j], f1[i], f2[i]):
                dominated[i] = True
                break
    return ~dominated


def fast_non_dominated_sort_maxmin(f1: np.ndarray, f2: np.ndarray) -> np.ndarray:
    """
    Returns Pareto ranks:
        rank 0: non-dominated front
        rank 1: dominated only by rank 0 points
        ...
    """
    n = len(f1)
    S = [[] for _ in range(n)]          # points dominated by i
    n_dom = np.zeros(n, dtype=int)      # number of points dominating i
    rank = np.full(n, -1, dtype=int)

    fronts: List[List[int]] = [[]]

    for p in range(n):
        for q in range(n):
            if p == q:
                continue
            if dominates_maxmin(f1[p], f2[p], f1[q], f2[q]):
                S[p].append(q)
            elif dominates_maxmin(f1[q], f2[q], f1[p], f2[p]):
                n_dom[p] += 1

        if n_dom[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    i = 0
    while i < len(fronts) and len(fronts[i]) > 0:
        next_front = []
        for p in fronts[i]:
            for q in S[p]:
                n_dom[q] -= 1
                if n_dom[q] == 0:
                    rank[q] = i + 1
                    next_front.append(q)
        i += 1
        fronts.append(next_front)

    return rank


def hypervolume_2d_maxmin(f1: np.ndarray, f2: np.ndarray, ref: Tuple[float, float]) -> float:
    """
    2D hypervolume for maximise f1, minimise f2.
    Reference ref = (f1_ref low, f2_ref high), a "worst" point.
    """
    mask = pareto_mask_maxmin(f1, f2)
    p1 = f1[mask]
    p2 = f2[mask]

    if len(p1) == 0:
        return 0.0

    order = np.argsort(p2)  # by cost ascending
    p1 = p1[order]
    p2 = p2[order]

    f1_ref, f2_ref = ref
    hv = 0.0
    best_f1 = f1_ref

    for i in range(len(p1)):
        width = max(0.0, p1[i] - best_f1)
        height = max(0.0, f2_ref - p2[i])
        hv += width * height
        best_f1 = max(best_f1, p1[i])

    return float(hv)


def hypervolume_contributions_2d_maxmin(f1: np.ndarray, f2: np.ndarray, ref: Tuple[float, float]) -> np.ndarray:
    """
    Hypervolume contribution of each Pareto point:
        contrib_i = HV(all Pareto points) - HV(all Pareto points except i)

    Returns an array aligned to original indices, with 0 for non-Pareto points.
    """
    mask = pareto_mask_maxmin(f1, f2)
    idx_p = np.where(mask)[0]
    contrib = np.zeros(len(f1), dtype=float)

    if len(idx_p) == 0:
        return contrib

    hv_all = hypervolume_2d_maxmin(f1, f2, ref=ref)

    for i in idx_p:
        keep = np.ones(len(f1), dtype=bool)
        keep[i] = False
        hv_minus = hypervolume_2d_maxmin(f1[keep], f2[keep], ref=ref)
        contrib[i] = max(0.0, hv_all - hv_minus)

    return contrib


def epsilon_constraint_best(f1: np.ndarray, f2: np.ndarray, eps_list: List[float]) -> pd.DataFrame:
    """
    For each cost threshold eps, find the design with minimal cost <= eps
    that gives maximum f1 (performance) among feasible points.

    Output:
        eps, best_perf, cost, index
    """
    rows = []
    for eps in eps_list:
        feasible = f2 <= eps
        if not np.any(feasible):
            rows.append({"epsilon_cost": eps, "best_perf": np.nan, "cost": np.nan, "index": -1})
            continue
        i_best = int(np.argmax(f1[feasible]))
        idx = int(np.where(feasible)[0][i_best])
        rows.append({"epsilon_cost": eps, "best_perf": float(f1[idx]), "cost": float(f2[idx]), "index": idx})
    return pd.DataFrame(rows)


# GP models


def make_gp(seed: int = 42) -> GaussianProcessRegressor:
    kernel = (
        ConstantKernel(1.0, (1e-2, 1e2))
        * RBF(length_scale=np.ones(2), length_scale_bounds=(1e-2, 1e2))
        + WhiteKernel(noise_level=1e-2, noise_level_bounds=(1e-6, 1e1))
    )
    return GaussianProcessRegressor(
        kernel=kernel,
        normalize_y=True,
        random_state=seed,
        n_restarts_optimizer=4,
    )


# MOBO acquisition + Batch selection


def propose_batch_mobo(
    rng: np.random.Generator,
    gp1: GaussianProcessRegressor,
    gp2: GaussianProcessRegressor,
    bounds: Tuple[Tuple[float, float], Tuple[float, float]],
    batch_size: int = 4,
    n_candidates: int = 12000,
    explore_bonus: float = 0.18,
    min_sep: float = 0.12,
) -> Tuple[np.ndarray, List[Dict]]:
    """
    Batch MOBO via random scalarisation with greedy diversity:

    score = w*m1 - (1-w)*m2 + explore_bonus*unc

    Then select top points greedily subject to a minimum separation
    in design space (to avoid redundant batch proposals).
    """
    (a1, b1), (a2, b2) = bounds
    Xcand = np.column_stack([
        rng.uniform(a1, b1, size=n_candidates),
        rng.uniform(a2, b2, size=n_candidates),
    ])

    mu1, std1 = gp1.predict(Xcand, return_std=True)
    mu2, std2 = gp2.predict(Xcand, return_std=True)

    m1 = (mu1 - np.mean(mu1)) / (np.std(mu1) + 1e-12)
    m2 = (mu2 - np.mean(mu2)) / (np.std(mu2) + 1e-12)

    unc = (std1 / (np.mean(std1) + 1e-12)) + (std2 / (np.mean(std2) + 1e-12))

    # sample several scalarisations, take union of top candidates
    # (gives better Pareto coverage)
    scalar_weights = rng.uniform(0.10, 0.90, size=max(8, batch_size * 3))

    # Score each candidate as max over sampled scalarisations
    all_scores = np.full(n_candidates, -np.inf, dtype=float)
    all_w = np.zeros(n_candidates, dtype=float)

    for w in scalar_weights:
        score = w * m1 - (1.0 - w) * m2 + explore_bonus * unc
        better = score > all_scores
        all_scores[better] = score[better]
        all_w[better] = w

    # Greedy selection with diversity constraint
    order = np.argsort(-all_scores)
    chosen = []
    chosen_meta = []

    for idx in order:
        if len(chosen) >= batch_size:
            break
        x = Xcand[idx]

        if len(chosen) == 0:
            chosen.append(x)
            chosen_meta.append({"w": float(all_w[idx]), "score": float(all_scores[idx])})
            continue

        # ensure separation from already chosen
        dists = [float(np.linalg.norm(x - c)) for c in chosen]
        if min(dists) >= min_sep:
            chosen.append(x)
            chosen_meta.append({"w": float(all_w[idx]), "score": float(all_scores[idx])})

    # if not enough, relax separation
    relax = min_sep
    while len(chosen) < batch_size:
        relax *= 0.85
        for idx in order:
            if len(chosen) >= batch_size:
                break
            x = Xcand[idx]
            dists = [float(np.linalg.norm(x - c)) for c in chosen] if chosen else [np.inf]
            if min(dists) >= relax:
                chosen.append(x)
                chosen_meta.append({"w": float(all_w[idx]), "score": float(all_scores[idx])})
        if relax < 0.02:
            break

    Xnext = np.array(chosen, dtype=float)
    return Xnext, chosen_meta


# Main experiment


def run_mobo_plus(
    outdir: str,
    seed: int = 42,
    budget: int = 100,
    init_points: int = 16,
    batch_size: int = 4,
    n_candidates: int = 12000,
    explore_bonus: float = 0.18,
    shift_test: bool = True,
) -> None:
    rng = set_seed(seed)
    ensure_outdir(outdir)

    cfg = SimSettings()
    bounds = (cfg.x1_bounds, cfg.x2_bounds)

    # Initialise with LHS
    X = latin_hypercube_2d(rng, init_points, bounds)
    f1, f2 = simulator_multiobjective(rng, X, cfg, noisy=True, shift=0.0)

    # Hypervolume reference point (worse-than-observed)
    f1_ref = float(np.min(f1) - 0.75)
    f2_ref = float(np.max(f2) + 0.75)
    hv_ref = (f1_ref, f2_ref)

    trace = []
    for i in range(init_points):
        trace.append({
            "iter": i,
            "x1": float(X[i, 0]),
            "x2": float(X[i, 1]),
            "f1_perf": float(f1[i]),
            "f2_cost": float(f2[i]),
            "phase": "init",
            "batch_id": -1,
        })

    gp1 = make_gp(seed=seed)
    gp2 = make_gp(seed=seed)

    hv_curve = []
    step = init_points
    batch_counter = 0

    # Loop in batches
    while step < budget:
        gp1.fit(X, f1)
        gp2.fit(X, f2)

        hv_now = hypervolume_2d_maxmin(f1, f2, ref=hv_ref)
        hv_curve.append({"iter": step, "hypervolume": hv_now})

        # propose batch
        Xnext, meta_list = propose_batch_mobo(
            rng=rng,
            gp1=gp1,
            gp2=gp2,
            bounds=bounds,
            batch_size=min(batch_size, budget - step),
            n_candidates=n_candidates,
            explore_bonus=explore_bonus,
            min_sep=0.12,
        )

        f1_next, f2_next = simulator_multiobjective(rng, Xnext, cfg, noisy=True, shift=0.0)

        # append batch
        X = np.vstack([X, Xnext])
        f1 = np.concatenate([f1, f1_next])
        f2 = np.concatenate([f2, f2_next])

        # log trace
        for k in range(len(Xnext)):
            trace.append({
                "iter": step + k,
                "x1": float(Xnext[k, 0]),
                "x2": float(Xnext[k, 1]),
                "f1_perf": float(f1_next[k]),
                "f2_cost": float(f2_next[k]),
                "phase": "mobo_batch",
                "batch_id": batch_counter,
                "scalar_weight_w": meta_list[k].get("w", np.nan),
                "acq_score": meta_list[k].get("score", np.nan),
                "hypervolume_before_batch": hv_now,
            })

        step += len(Xnext)
        batch_counter += 1

    # Final HV
    hv_final = hypervolume_2d_maxmin(f1, f2, ref=hv_ref)
    hv_curve.append({"iter": budget, "hypervolume": hv_final})
    hv_df = pd.DataFrame(hv_curve)

    # Pareto mask + ranks + HV contributions
    pmask = pareto_mask_maxmin(f1, f2)
    ranks = fast_non_dominated_sort_maxmin(f1, f2)
    hv_contrib = hypervolume_contributions_2d_maxmin(f1, f2, ref=hv_ref)

    # Save main trace
    trace_df = pd.DataFrame(trace)
    trace_df.to_csv(os.path.join(outdir, "tables", "mobo_plus_trace.csv"), index=False)
    hv_df.to_csv(os.path.join(outdir, "tables", "hypervolume_trace.csv"), index=False)

    # Pareto table
    pareto_idx = np.where(pmask)[0]
    pareto_df = pd.DataFrame({
        "index": pareto_idx,
        "x1": X[pmask, 0],
        "x2": X[pmask, 1],
        "f1_perf": f1[pmask],
        "f2_cost": f2[pmask],
        "pareto_rank": ranks[pmask],
        "hv_contribution": hv_contrib[pmask],
    }).sort_values(["f2_cost", "f1_perf"], ascending=[True, False]).reset_index(drop=True)

    pareto_df.to_csv(os.path.join(outdir, "tables", "pareto_front_plus.csv"), index=False)

    # Epsilon-constraint report: pick best perf under cost thresholds
    cost_min, cost_max = float(np.min(f2)), float(np.max(f2))
    eps_list = list(np.linspace(cost_min + 0.05, min(cost_max, cost_min + 1.8), 10))
    eps_df = epsilon_constraint_best(f1, f2, eps_list=eps_list)
    eps_df.to_csv(os.path.join(outdir, "tables", "epsilon_constraint_report.csv"), index=False)

    # Optional OOD shift check on selected Pareto points (noiseless)
    shift_report = None
    if shift_test:
        # evaluate top Pareto points by hypervolume contribution
        topk = min(6, len(pareto_df))
        if topk > 0:
            cand_idx = pareto_df.sort_values("hv_contribution", ascending=False).head(topk)["index"].to_numpy().astype(int)
            Xcand = X[cand_idx]
            f1_in, f2_in = simulator_multiobjective(rng, Xcand, cfg, noisy=False, shift=0.0)
            f1_shift, f2_shift = simulator_multiobjective(rng, Xcand, cfg, noisy=False, shift=0.35)

            shift_report = []
            for i, idx in enumerate(cand_idx):
                shift_report.append({
                    "index": int(idx),
                    "x1": float(X[idx, 0]),
                    "x2": float(X[idx, 1]),
                    "f1_in": float(f1_in[i]),
                    "f1_shift": float(f1_shift[i]),
                    "f1_drop": float(f1_in[i] - f1_shift[i]),
                    "f2_in": float(f2_in[i]),
                    "f2_shift": float(f2_shift[i]),
                })

    # Figures
    figdir = os.path.join(outdir, "figures")

    # Pareto scatter: cost vs performance
    plt.figure(figsize=(7.2, 5.4))
    plt.scatter(f2, f1, s=18, label="All evaluated")
    plt.scatter(pareto_df["f2_cost"], pareto_df["f1_perf"], s=45, label="Pareto front")
    plt.xlabel("Cost (minimise)")
    plt.ylabel("Performance (maximise)")
    plt.title("Pareto Front (MOBO++)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "pareto_front.png"), dpi=220)
    plt.close()

    # Pareto ranks visualization: colour by rank using separate plot per rank groups (simple)
    plt.figure(figsize=(7.2, 5.4))
    for r in np.unique(ranks):
        idx = np.where(ranks == r)[0]
        if len(idx) == 0:
            continue
        plt.scatter(f2[idx], f1[idx], s=18, label=f"rank {r}")
        if r >= 4:
            break  # keep plot readable
    plt.xlabel("Cost (minimise)")
    plt.ylabel("Performance (maximise)")
    plt.title("Non-dominated Sorting Levels (Pareto Ranks)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "pareto_ranks.png"), dpi=220)
    plt.close()

    # Hypervolume curve
    plt.figure(figsize=(7.0, 4.8))
    plt.plot(hv_df["iter"], hv_df["hypervolume"], marker="o", linewidth=1.2)
    plt.xlabel("Evaluations")
    plt.ylabel("Hypervolume (higher is better)")
    plt.title("Hypervolume Improvement over Evaluations")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "hypervolume_curve.png"), dpi=220)
    plt.close()

    # Hypervolume contribution bar (top Pareto points)
    top_contrib = pareto_df.sort_values("hv_contribution", ascending=False).head(min(12, len(pareto_df))).copy()
    if len(top_contrib) > 0:
        plt.figure(figsize=(8.2, 4.8))
        plt.bar([str(int(i)) for i in top_contrib["index"]], top_contrib["hv_contribution"])
        plt.xlabel("Pareto point index")
        plt.ylabel("Hypervolume contribution")
        plt.title("Most Valuable Pareto Points (Hypervolume Contribution)")
        plt.tight_layout()
        plt.savefig(os.path.join(figdir, "hv_contribution_top.png"), dpi=220)
        plt.close()

    # Epsilon-constraint curve
    plt.figure(figsize=(7.0, 4.8))
    plt.plot(eps_df["epsilon_cost"], eps_df["best_perf"], marker="o", linewidth=1.2)
    plt.xlabel("Cost threshold ε")
    plt.ylabel("Best feasible performance")
    plt.title("Epsilon-Constraint Trade-off Curve")
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "epsilon_constraint_curve.png"), dpi=220)
    plt.close()

    # Design space plot
    plt.figure(figsize=(7.2, 5.4))
    plt.scatter(X[:, 0], X[:, 1], s=18, label="All evaluated")
    plt.scatter(pareto_df["x1"], pareto_df["x2"], s=45, label="Pareto designs")
    plt.xlabel("x1")
    plt.ylabel("x2")
    plt.title("Design Space Coverage + Pareto Designs")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(figdir, "design_space_pareto.png"), dpi=220)
    plt.close()

    # Save metrics JSON
    metrics = {
        "seed": seed,
        "budget": budget,
        "init_points": init_points,
        "batch_size": batch_size,
        "n_candidates": n_candidates,
        "explore_bonus": explore_bonus,
        "hv_reference_point": {"f1_ref": hv_ref[0], "f2_ref": hv_ref[1]},
        "hypervolume_final": float(hv_final),
        "n_pareto_points": int(len(pareto_df)),
        "best_perf_overall": float(np.max(f1)),
        "min_cost_overall": float(np.min(f2)),
        "pareto_best_perf": float(np.max(pareto_df["f1_perf"])) if len(pareto_df) else np.nan,
        "pareto_min_cost": float(np.min(pareto_df["f2_cost"])) if len(pareto_df) else np.nan,
        "ood_shift_check_top_pareto": shift_report,
        "gp_kernels": {
            "gp1_perf": str(gp1.kernel_),
            "gp2_cost": str(gp2.kernel_),
        },
    }
    save_json(metrics, os.path.join(outdir, "metrics_report.json"))

    # Print summary
    print("\n=== MOBO++ (Pareto + Ranks + HV contribution + Epsilon constraint + Batch BO) ===")
    print(f"Outputs saved to: {os.path.abspath(outdir)}\n")
    print("Final hypervolume:", float(hv_final))
    print("Pareto front size:", int(len(pareto_df)))
    print("Best performance observed:", float(np.max(f1)))
    print("Minimum cost observed:", float(np.min(f2)))
    if len(pareto_df) > 0:
        top = pareto_df.sort_values("hv_contribution", ascending=False).head(1).iloc[0]
        print("\nTop Pareto point by hypervolume contribution:")
        print("  index:", int(top["index"]))
        print("  x1,x2:", float(top["x1"]), float(top["x2"]))
        print("  perf:", float(top["f1_perf"]), " cost:", float(top["f2_cost"]))
        print("  hv contrib:", float(top["hv_contribution"]))

    if shift_report is not None and len(shift_report) > 0:
        print("\nOOD shift check (top Pareto points, noiseless):")
        worst = max(shift_report, key=lambda d: d["f1_drop"])
        print("  worst perf drop:", float(worst["f1_drop"]), "at index", int(worst["index"]))

    print("\nFigures ->", os.path.join(outdir, "figures"))
    print("Tables  ->", os.path.join(outdir, "tables"), "\n")


def make_argparser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(description="UH MOBO++ PoC")
    p.add_argument("--outdir", type=str, default="uh_mobo_plus", help="Output directory")
    p.add_argument("--seed", type=int, default=42, help="Random seed")
    p.add_argument("--budget", type=int, default=100, help="Total evaluations (including init)")
    p.add_argument("--init_points", type=int, default=16, help="Initial LHS points")
    p.add_argument("--batch_size", type=int, default=4, help="Batch size per MOBO iteration")
    p.add_argument("--n_candidates", type=int, default=12000, help="Random candidates per batch")
    p.add_argument("--explore_bonus", type=float, default=0.18, help="Exploration strength")
    p.add_argument("--no_shift_test", action="store_true", help="Disable OOD shift check")
    return p


def main() -> None:
    parser = make_argparser()
    args, _ = parser.parse_known_args()  # Robust in Jupyter + terminal (ignores -f kernel.json)
    run_mobo_plus(
        outdir=args.outdir,
        seed=args.seed,
        budget=args.budget,
        init_points=args.init_points,
        batch_size=args.batch_size,
        n_candidates=args.n_candidates,
        explore_bonus=args.explore_bonus,
        shift_test=(not args.no_shift_test),
    )


if __name__ == "__main__":
    main()


=== MOBO++ (Pareto + Ranks + HV contribution + Epsilon constraint + Batch BO) ===
Outputs saved to: /Users/petalc01/Data Science Hertfordshire/uh_mobo_plus

Final hypervolume: 83.85202924240279
Pareto front size: 15
Best performance observed: 1.3893459905783005
Minimum cost observed: -0.04151575088423652

Top Pareto point by hypervolume contribution:
  index: 89
  x1,x2: -0.01860984679671196 0.1094652212937901
  perf: 0.13141950343637387  cost: -0.04151575088423652
  hv contrib: 0.29874602106409043

OOD shift check (top Pareto points, noiseless):
  worst perf drop: -0.5002095879411976 at index 83

Figures -> uh_mobo_plus/figures
Tables  -> uh_mobo_plus/tables 

